In [1]:
import pandas as pd
import numpy as np
import heapq
import time
from collections import defaultdict, deque
import sys
import itertools
import os 
import time as timer
import math
from bisect import bisect_left, bisect # For efficient searching in sorted lisst
import multiprocessing as mp

# from file_management import read_capacity, read_market

In [2]:
def read_capacity(CAPACITY_FILE, airport_substitutions=None):
    capacity = pd.read_csv(CAPACITY_FILE, sep=',')

    capacity['deptime'] = pd.to_datetime(capacity['deptime'])
    capacity['arrtime'] = pd.to_datetime(capacity['arrtime'])

    # Map Weekday_Z to day number (0=Mon, 6=Sun)
    day_map = {'Mon': 0, 'Tue': 1, 'Wed': 2, 'Thu': 3, 'Fri': 4, 'Sat': 5, 'Sun': 6}
    capacity['day'] = capacity['Weekday_Z'].map(day_map)

    # Calculate total minutes from start of week (Monday 00:00) # Dep Time = Day * 1440 + Hour * 60 + Minute
    capacity['dep_time'] = capacity['day'] * 1440 + capacity['deptime'].dt.hour * 60 + capacity['deptime'].dt.minute
    capacity['DD_Z'] = capacity['DD_Z'].fillna(0)
    capacity['arr_time'] = capacity['day'] * 1440 + capacity['arrtime'].dt.hour * 60 + capacity['arrtime'].dt.minute + capacity['DD_Z']*1440

    # --- Column Renaming and Selection ---
    rename_columns = {'Net Payload': 'cap_kg','Net Volume': 'cap_m3','Orig': 'ori',
        'Dest': 'des','Flight Number': 'flight_number','A/C': 'aircraft_type'}
    capacity = capacity.rename(columns=rename_columns)

    # Define desired columns
    columns = [ 'ori', 'des', 'dep_time', 'arr_time', 'cap_kg']  # , 'aircraft_type', 'flight_number',
    capacity = capacity[[col for col in columns if col in capacity.columns]]

    # CONVERT SAME AIRPORTS FROM DICTIONARY - same_airports - CONVERT EVERY KEY TO ITS VALUE
    for key, value in airport_substitutions.items():
        capacity.loc[capacity['ori'] == key, 'ori'] = value
        capacity.loc[capacity['des'] == key, 'des'] = value

    capacity['dep_time'] = capacity['dep_time'].astype(int) # TIME SINCE START OF SIMULATION
    capacity['arr_time'] = capacity['arr_time'].astype(int) # TIME SINCE START OF SIMULATION
    # capacity['day'] = capacity['day'].astype(int) # DAY OF SIMULATION OF DEPARTURE FLIGHT
    capacity['flight_id'] = capacity.index # FLIGHT NUMBER
    print(f"Capacity data read: {len(capacity)} rows.")
    return capacity


def read_market(MARKET_FILE, airport_substitutions=None):
    market = pd.read_csv(MARKET_FILE, sep=';')

    market = market.rename(columns={'origin': 'ori', 'destination': 'des', 'Market CHW': 'demand', 'Day': 'day'})
    if 'product' in market.columns: market.drop(columns=['product'], inplace=True)
    if 'Market Allin Yield' in market.columns: market.drop(columns=['Market Allin Yield'], inplace=True)

    day_map = {'Mon': 0, 'Tue': 1, 'Wed': 2, 'Thu': 3, 'Fri': 4, 'Sat': 5, 'Sun': 6}
    market['day'] = market['day'].map(day_map)

    # convert HH:MM to minutes
    time_minutes = market['Time'].str.split(':', expand=True).astype(int).apply(lambda x: x[0] * 60 + x[1], axis=1)
    market['time'] = market['day'] * 1440 + time_minutes

    # SUBSITUTE AIRPORTS FROM same_airports DICTIONERY - CONVERT EVERY KEY TO ITS VALUE
    for key, value in airport_substitutions.items():
        market.loc[market['ori'] == key, 'ori'] = value
        market.loc[market['des'] == key, 'des'] = value

    # Add original key
    market['ODT'] = market['ori'] + '/' + market['des'] + '/' + market['time'].astype(str)

    # WE CREATED DUPLICATES KEY AS A RESULT OF CONVERTING SAME AIRPORTS
    # MERGE THE DEMAND VALUES FOR THE SAME KEY
    market = market.groupby(['ODT', 'ori', 'des', 'time']).agg({'demand': 'sum'}).reset_index()
    market = market[['ori', 'des', 'demand', 'time', 'ODT']].copy()

    # Convert types
    market['time'] = market['time'].astype(int) # TIME SINCE START OF SIMULATION
    # market['day'] = market['day'].astype(int) # DAY OF SIMULATION
    market['demand'] = pd.to_numeric(market['demand'], errors='coerce').fillna(0)
    print(f"Market data read: {len(market)} rows.")
    return market

In [3]:
K_PATHS = 5

MIN_CONNECT_MINS = 60
MAX_CONNECT_WAIT_MINS = 2 * 24 * 60 # 48 hours - Aggressive Pruning
MAX_TOTAL_DURATION_MINS = 5 * 24 * 60 # Pruning: Max overall path duration 

DIVERSITY_NODE_PENALTY = 1.2 # Small multiplicative penalty for reusing
DIVERSITY_FLIGHT_PENALTY = 2.0 # Larger penalty for reusing exact flight IDs
A_STAR_HEURISTIC_FACTOR = 0 # Factor for simple heuristic (0 = Dijkstra) - 

CAPACITY_FILE = 'files/capacity.csv'
MARKET_FILE = 'files/market.csv'

airport_substitutions = {
    'DOH': "BAH",
    'SZX': "HKG",
    'CAN': "HKG",
    'STN': 'LHR',
    'LTN': 'LHR'
}

# schedule_df_raw = read_capacity(CAPACITY_FILE, airport_substitutions=airport_substitutions)
# demand_df_raw = read_market(MARKET_FILE, airport_substitutions=airport_substitutions)

tie_breaker = itertools.count() # removed for paralell

In [4]:
# --- Tuned Fast Network Class ---
class TunedFastNetwork:
    def __init__(self, schedule_df, demand_df, k_paths, min_connect_mins, max_wait_mins, max_total_duration):
        print(f"\nInitializing Tuned Fast Network (MaxWait={max_wait_mins}, MaxDuration={max_total_duration})...")
        if schedule_df.empty or demand_df.empty: raise ValueError("Input data is empty.")
        self.k_paths = k_paths
        self.min_connect_mins = min_connect_mins
        self.max_wait_mins = max_wait_mins
        self.max_total_duration = max_total_duration # New pruning limit

        # Lookups
        self.flight_data = schedule_df.set_index('flight_id').to_dict('index')
        self.demand_details = demand_df.set_index('ODT').to_dict('index')
        self.flights_by_origin = defaultdict(lambda: {'ids': [], 'dep_times': []})
        for _, row in schedule_df.sort_values(by='dep_time').iterrows():
             key = row['ori']; self.flights_by_origin[key]['ids'].append(row['flight_id']); self.flights_by_origin[key]['dep_times'].append(row['dep_time'])

        # Graph
        self.graph = defaultdict(list)
        self._build_graph()

    def _add_edge(self, u, v, elapsed_time, flight_id=None, capacity=float('inf'), is_flight=False):
        if elapsed_time < 0: return
        cap = float(capacity) if math.isfinite(capacity) else float('inf')
        self.graph[u].append({'neighbor': v, 'weight': float(elapsed_time), 'flight_id': flight_id,'capacity': cap, 'is_flight': is_flight})

    def _find_first_flight_idx_after(self, dep_times_list, min_departure_time):
        idx = bisect_left(dep_times_list, min_departure_time)
        return idx if idx < len(dep_times_list) else -1

    def _build_graph(self):
        print("Building graph...")
        start_time = timer.time()
        # Simplified node types: ('S', odt), ('D', airport), (fid, 'd'), (fid, 'a')

        for fid, f in self.flight_data.items(): # Flight Edges
            self._add_edge((fid, 'd'), (fid, 'a'), f['arr_time'] - f['dep_time'], fid, f['cap_kg'], True)

        for fid, f in self.flight_data.items(): # Connections & Sinks
            arr_node, arr_time, arr_apt = (fid, 'a'), f['arr_time'], f['des']
            self._add_edge(arr_node, ('D', arr_apt), 0) # Sink Edge
            origin_data = self.flights_by_origin.get(arr_apt)
            if origin_data:
                idx = self._find_first_flight_idx_after(origin_data['dep_times'], arr_time + self.min_connect_mins)
                if idx != -1:
                    for i in range(idx, len(origin_data['ids'])):
                        wait = origin_data['dep_times'][i] - arr_time
                        if wait <= self.max_wait_mins: self._add_edge(arr_node, (origin_data['ids'][i], 'd'), wait)
                        else: break

        for odt, dem in self.demand_details.items(): # Source Edges
            origin_data = self.flights_by_origin.get(dem['ori'])
            if origin_data:
                idx = self._find_first_flight_idx_after(origin_data['dep_times'], dem['time'])
                if idx != -1:
                    for i in range(idx, len(origin_data['ids'])):
                        wait = origin_data['dep_times'][i] - dem['time']
                        if wait <= self.max_wait_mins: self._add_edge(('S', odt), (origin_data['ids'][i], 'd'), wait)
                        else: break
        print(f"Graph built in {timer.time() - start_time:.2f}s")


    def _a_star(self, start_node, end_node, node_penalties=None, flight_penalties=None):
        """A* Search incorporating node AND flight penalties + max duration pruning."""
        global tie_breaker
        if node_penalties is None: node_penalties = defaultdict(lambda: 1.0)
        if flight_penalties is None: flight_penalties = defaultdict(lambda: 1.0)
        heuristic = lambda u, v: 0.0

        pq = [(heuristic(start_node, end_node), next(tie_breaker), 0.0, start_node, [{'node': start_node, 'edge_info': None}])]
        visited_g_costs = defaultdict(lambda: float('inf'))
        visited_g_costs[start_node] = 0.0

        while pq:
            f, _, g, curr_node, path = heapq.heappop(pq)

            # --- Pruning based on Max Duration ---
            if g > self.max_total_duration: continue
            # ---

            if g > visited_g_costs[curr_node]: continue
            if curr_node == end_node: return g, path

            if curr_node not in self.graph: continue

            current_node_penalty = node_penalties[curr_node]

            for edge in self.graph[curr_node]:
                neighbor = edge['neighbor']
                # Combine Node and Flight Penalties
                # Penalize based on *current* node and the *specific flight* of the edge (if any)
                flight_id = edge['flight_id']
                edge_flight_penalty = flight_penalties[flight_id] if flight_id is not None else 1.0
                combined_penalty = current_node_penalty * edge_flight_penalty

                cost_to_neighbor = edge['weight'] * combined_penalty
                new_g = g + cost_to_neighbor

                if new_g < visited_g_costs[neighbor]:
                    visited_g_costs[neighbor] = new_g
                    h = heuristic(neighbor, end_node)
                    new_f = new_g + h
                    new_step = {'node': neighbor, 'edge_info': edge}
                    heapq.heappush(pq, (new_f, next(tie_breaker), new_g, neighbor, path + [new_step]))
        return float('inf'), None

    def _format_path(self, path_list):
        """Formats path list, calculates ORIGINAL cost and capacity."""
        if not path_list: return None
        flight_ids = []
        min_capacity = float('inf')
        total_original_time = 0.0 # Sum original weights
        for step in path_list[1:]:
            edge = step['edge_info']
            if edge:
                total_original_time += edge['weight'] # Sum original weights
                if edge['is_flight']:
                    flight_ids.append(edge['flight_id'])
                    min_capacity = min(min_capacity, edge['capacity'])
            if isinstance(step['node'], tuple) and step['node'][0] == 'D': break
        min_cap = 0.0 if min_capacity == float('inf') else min_capacity
        # Return the true duration calculated from original weights
        return {'total_time': total_original_time, 'flight_ids': flight_ids, 'min_capacity': min_cap}

    def find_k_paths(self, node_penalty=DIVERSITY_NODE_PENALTY, flight_penalty=DIVERSITY_FLIGHT_PENALTY):
        """Finds K paths using A* and combined node/flight penalization."""
        print(f"\nFinding {self.k_paths} paths (NodePen={node_penalty}, FlightPen={flight_penalty})...")
        all_results = {}
        total_demands = len(self.demand_details)
        start_run_time = timer.time()

        for i, (odt_id, demand) in enumerate(self.demand_details.items()):
            if (i + 1) % 500 == 0:
                elapsed = timer.time() - start_run_time; rate = (i+1)/elapsed if elapsed > 0 else 0
                print(f"  Processed {i+1}/{total_demands}... ({rate:.1f} req/s)")

            start_node = ('S', odt_id)
            target_node = ('D', demand['des'])

            found_paths_output = [] # Store final formatted paths
            node_penalties = defaultdict(lambda: 1.0)
            flight_penalties = defaultdict(lambda: 1.0)
            found_sequences = set()

            for k in range(self.k_paths):
                # Pass current penalties to A*
                cost_internal, path_list = self._a_star(start_node, target_node, node_penalties, flight_penalties)

                if path_list:
                    nodes_tuple = tuple(s['node'] for s in path_list)
                    if nodes_tuple not in found_sequences:
                        # Format using original weights to get true time
                        formatted_path = self._format_path(path_list)
                        if formatted_path:
                            found_paths_output.append(formatted_path)
                            found_sequences.add(nodes_tuple)

                            # Apply penalties for next iteration
                            for step in path_list[1:]:
                                node = step['node']
                                edge = step['edge_info']
                                # Don't penalize source/sink nodes themselves heavily
                                if isinstance(node, tuple) and node[0] in ['S', 'D']: continue

                                # Penalize intermediate nodes visited
                                node_penalties[node] *= node_penalty

                                # Penalize specific flight IDs used
                                if edge and edge['is_flight']:
                                    flight_id = edge['flight_id']
                                    flight_penalties[flight_id] *= flight_penalty
                        else: break
                    else: break
                else: break

            # Sort the final collected paths by their true total time
            found_paths_output.sort(key=lambda p: p['total_time'])
            all_results[odt_id] = found_paths_output

        print(f"\nFinished finding paths in {timer.time() - start_run_time:.2f} seconds.")
        return all_results

# --- Main Execution Logic ---
print("Starting Tuned Fast Flight Path Analysis...")
# 1. Read Data
schedule_data = read_capacity(CAPACITY_FILE, airport_substitutions)
demand_data = read_market(MARKET_FILE, airport_substitutions)

# 2. Initialize and Run

# Note: Pass the new max_total_duration parameter
flight_network = TunedFastNetwork(
    schedule_data, demand_data, K_PATHS, MIN_CONNECT_MINS,
    MAX_CONNECT_WAIT_MINS, MAX_TOTAL_DURATION_MINS
)
# Use the find_k_paths method
k_shortest_paths = flight_network.find_k_paths()

# 3. Output Summary
print("\n--- Pathfinding Summary ---")
total_paths_found = sum(len(p) for p in k_shortest_paths.values())
num_odts = len(k_shortest_paths)
avg_paths = total_paths_found / num_odts if num_odts > 0 else 0
print(f"Found results for {num_odts} ODTs.")
print(f"Total paths found: {total_paths_found}")
print(f"Average paths per ODT: {avg_paths:.2f} (Target K={K_PATHS})")

# Example ODT output (optional)
if num_odts > 0:
    example_odt_list = list(k_shortest_paths.keys())
    if example_odt_list:
        first_odt = example_odt_list[0]
        print(f"\nExample for ODT '{first_odt}':")
        if k_shortest_paths[first_odt]:
            for i, path in enumerate(k_shortest_paths[first_odt]):
                print(f"  Path {i+1}: Time={path['total_time']:.0f}, Cap={path['min_capacity']:.2f}, Flights={path['flight_ids']}")
        else:
            print("  No paths found for this ODT.")


Starting Tuned Fast Flight Path Analysis...
Capacity data read: 2057 rows.
Market data read: 3185 rows.

Initializing Tuned Fast Network (MaxWait=2880, MaxDuration=7200)...
Building graph...
Graph built in 0.07s

Finding 5 paths (NodePen=1.2, FlightPen=2.0)...
  Processed 500/3185... (281.3 req/s)
  Processed 1000/3185... (317.5 req/s)
  Processed 1500/3185... (296.2 req/s)
  Processed 2000/3185... (275.2 req/s)
  Processed 2500/3185... (263.8 req/s)
  Processed 3000/3185... (271.4 req/s)

Finished finding paths in 11.83 seconds.

--- Pathfinding Summary ---
Found results for 3185 ODTs.
Total paths found: 8710
Average paths per ODT: 2.73 (Target K=5)

Example for ODT 'AMS/LAX/1080':
  Path 1: Time=1850, Cap=47650.00, Flights=[24, 1306, 1685, 536]
  Path 2: Time=1850, Cap=46000.00, Flights=[25, 1307, 1177, 536]
  Path 3: Time=2110, Cap=46000.00, Flights=[25, 1317, 1116]
  Path 4: Time=2310, Cap=46000.00, Flights=[24, 1302, 347, 1313, 1846]
  Path 5: Time=2345, Cap=46000.00, Flights=[25,

In [101]:
# --- Further Simplified Network Class ---
class FastFlightNetwork:
    def __init__(self, schedule_df, demand_df, k_paths, min_connect_mins, max_wait_mins):
        print(f"\nInitializing Fast Network (MaxWait={max_wait_mins} min)...")
        if schedule_df.empty or demand_df.empty: raise ValueError("Input data is empty.")
        self.k_paths = k_paths
        self.min_connect_mins = min_connect_mins
        self.max_wait_mins = max_wait_mins

        # --- Lookups ---
        self.flight_data = schedule_df.set_index('flight_id').to_dict('index')
        self.demand_details = demand_df.set_index('ODT').to_dict('index')
        self.flights_by_origin = defaultdict(lambda: {'ids': [], 'dep_times': []})
        schedule_sorted = schedule_df.sort_values(by='dep_time')
        for idx, row in schedule_sorted.iterrows():
             key = row['ori']
             self.flights_by_origin[key]['ids'].append(row['flight_id'])
             self.flights_by_origin[key]['dep_times'].append(row['dep_time'])

        # --- Graph ---
        self.graph = defaultdict(list)
        self._build_graph()

    def _add_edge(self, u, v, elapsed_time, flight_id=None, capacity=float('inf'), is_flight=False):
        if elapsed_time < 0: return
        cap = float(capacity) if math.isfinite(capacity) else float('inf')
        self.graph[u].append({
            'neighbor': v, 'weight': float(elapsed_time), 'flight_id': flight_id,
            'capacity': cap, 'is_flight': is_flight
        })

    def _find_first_flight_idx_after(self, dep_times_list, min_departure_time):
        idx = bisect_left(dep_times_list, min_departure_time)
        return idx if idx < len(dep_times_list) else -1

    def _build_graph(self):
        print("Building graph...")
        start_time = timer.time()
        # Simplified node types: S=Source, D=DestSink, F=FlightArr/Dep
        # ('S', odt), ('D', airport), (fid, 'd'), (fid, 'a')

        # 1. Flight Edges (Dep -> Arr)
        for fid, f in self.flight_data.items():
            self._add_edge((fid, 'd'), (fid, 'a'), f['arr_time'] - f['dep_time'], fid, f['cap_kg'], True)

        # 2. Connections & Sinks (Arr -> Dep/Sink)
        for fid, f in self.flight_data.items():
            arr_node, arr_time, arr_apt = (fid, 'a'), f['arr_time'], f['des']
            self._add_edge(arr_node, ('D', arr_apt), 0) # Simplified Sink Edge

            origin_data = self.flights_by_origin.get(arr_apt)
            if origin_data:
                min_dep_time = arr_time + self.min_connect_mins
                start_idx = self._find_first_flight_idx_after(origin_data['dep_times'], min_dep_time)
                if start_idx != -1:
                    for i in range(start_idx, len(origin_data['ids'])):
                        wait = origin_data['dep_times'][i] - arr_time
                        if wait <= self.max_wait_mins:
                            self._add_edge(arr_node, (origin_data['ids'][i], 'd'), wait)
                        else: break

        # 3. Source Edges (Source -> Dep)
        for odt, dem in self.demand_details.items():
            source_node = ('S', odt)
            origin_data = self.flights_by_origin.get(dem['ori'])
            if origin_data:
                start_idx = self._find_first_flight_idx_after(origin_data['dep_times'], dem['time'])
                if start_idx != -1:
                    for i in range(start_idx, len(origin_data['ids'])):
                        wait = origin_data['dep_times'][i] - dem['time']
                        if wait <= self.max_wait_mins:
                             self._add_edge(source_node, (origin_data['ids'][i], 'd'), wait)
                        else: break
        print(f"Graph built in {timer.time() - start_time:.2f}s")


    def _heuristic(self, node, target_node):
        """Simple A* heuristic. Currently 0 (Dijkstra)."""
        # Could estimate hops remaining, but needs target info propagation. Keep 0.
        # Heuristic must be based on the *node representation*, not external data easily.
        return 0.0 * A_STAR_HEURISTIC_FACTOR # Effectively Dijkstra


    def _a_star(self, start_node, end_node, node_penalties=None):
        """A* Search incorporating node penalties for diversity."""
        global tie_breaker
        if node_penalties is None: node_penalties = defaultdict(lambda: 1.0)

        heuristic = self._heuristic
        pq = [(heuristic(start_node, end_node), next(tie_breaker), 0.0, start_node, [{'node': start_node, 'edge_info': None}])]
        visited_g_costs = defaultdict(lambda: float('inf'))
        visited_g_costs[start_node] = 0.0

        while pq:
            f, _, g, curr_node, path = heapq.heappop(pq)
            if g > visited_g_costs[curr_node]: continue
            if curr_node == end_node: return g, path

            if curr_node not in self.graph: continue

            # Apply penalty for visiting the current node (affects cost to neighbors)
            current_node_penalty = node_penalties[curr_node]

            for edge in self.graph[curr_node]:
                neighbor = edge['neighbor']
                # Cost includes edge weight PLUS penalty for visiting the *current* node
                cost_to_neighbor = edge['weight'] * current_node_penalty
                new_g = g + cost_to_neighbor

                if new_g < visited_g_costs[neighbor]:
                    visited_g_costs[neighbor] = new_g
                    h = heuristic(neighbor, end_node)
                    new_f = new_g + h
                    new_step = {'node': neighbor, 'edge_info': edge}
                    heapq.heappush(pq, (new_f, next(tie_breaker), new_g, neighbor, path + [new_step]))
        return float('inf'), None


    def _format_path(self, path_list):
        """Formats path list, calculates cost and capacity."""
        if not path_list: return None
        flight_ids = []
        min_capacity = float('inf')
        total_elapsed_time = 0.0
        for step in path_list[1:]:
            edge = step['edge_info']
            if edge:
                total_elapsed_time += edge['weight'] # Sum original weights for true time
                if edge['is_flight']:
                    flight_ids.append(edge['flight_id'])
                    min_capacity = min(min_capacity, edge['capacity'])
            # Stop if we hit the conceptual sink
            if isinstance(step['node'], tuple) and step['node'][0] == 'D':
                 break
        min_cap = 0.0 if min_capacity == float('inf') else min_capacity
        return {'total_time': total_elapsed_time, 'flight_ids': flight_ids, 'min_capacity': min_cap}


    def find_k_paths(self, node_penalty_factor=DIVERSITY_NODE_PENALTY):
        """Finds K paths using A* and node penalization heuristic for diversity."""
        print(f"\nFinding {self.k_paths} paths per demand (Node Penalty={node_penalty_factor})...")
        all_results = {}
        total_demands = len(self.demand_details)
        start_run_time = timer.time()

        for i, (odt_id, demand) in enumerate(self.demand_details.items()):
            if (i + 1) % 500 == 0:
                elapsed = timer.time() - start_run_time; rate = (i+1)/elapsed if elapsed > 0 else 0
                print(f"  Processed {i+1}/{total_demands}... ({rate:.1f} req/s)")

            start_node = ('S', odt_id)
            target_node = ('D', demand['des']) # Use simplified destination sink

            found_paths = []
            # Node penalties accumulate over the K iterations for this ODT
            node_penalties = defaultdict(lambda: 1.0)
            found_sequences = set()

            for k in range(self.k_paths):
                # Pass current node penalties to A*
                cost, path_list = self._a_star(start_node, target_node, node_penalties)

                if path_list:
                    nodes_tuple = tuple(s['node'] for s in path_list)
                    if nodes_tuple not in found_sequences:
                        # Format uses original weights to get true time
                        formatted_path = self._format_path(path_list)
                        if formatted_path:
                            found_paths.append(formatted_path) # Store formatted path
                            found_sequences.add(nodes_tuple)

                            # Apply node penalties for next iteration
                            for step in path_list[1:]: # Don't penalize source
                                node = step['node']
                                # Penalize flight arrival/departure nodes and potentially sinks
                                if isinstance(node, tuple) and node[0] in ['D']: continue # Maybe don't penalize final sink
                                if isinstance(node, int) or (isinstance(node, tuple) and node[0] != 'S'):
                                     node_penalties[node] *= node_penalty_factor
                        else: break # Formatting failed?
                    else: break # Duplicate sequence found
                else: break # No more paths found

            all_results[odt_id] = found_paths # Already formatted

        print(f"\nFinished finding paths in {timer.time() - start_run_time:.2f} seconds.")
        return all_results

# --- Main Execution Logic ---
print("Starting Fast Flight Path Analysis...")
# 1. Read Data
schedule_data = read_capacity(CAPACITY_FILE, airport_substitutions)
demand_data = read_market(MARKET_FILE, airport_substitutions)

# 2. Initialize and Run

flight_network = FastFlightNetwork(
    schedule_data, demand_data, K_PATHS, MIN_CONNECT_MINS, MAX_CONNECT_WAIT_MINS
)
k_shortest_paths = flight_network.find_k_paths()

# 3. Output Summary
print("\n--- Pathfinding Summary ---")
total_paths_found = sum(len(p) for p in k_shortest_paths.values())
num_odts = len(k_shortest_paths)
avg_paths = total_paths_found / num_odts if num_odts > 0 else 0
print(f"Found results for {num_odts} ODTs.")
print(f"Total paths found: {total_paths_found}")
print(f"Average paths per ODT: {avg_paths:.2f} (Target K={K_PATHS})")

# Example ODT output (optional)
if num_odts > 0:
    first_odt = list(k_shortest_paths.keys())[0]
    print(f"\nExample for ODT '{first_odt}':")
    if k_shortest_paths[first_odt]:
        for i, path in enumerate(k_shortest_paths[first_odt]):
                print(f"  Path {i+1}: Time={path['total_time']:.0f}, Cap={path['min_capacity']:.2f}, Flights={path['flight_ids']}")
    else:
        print("  No paths found for this ODT.")


Starting Fast Flight Path Analysis...
Capacity data read: 2057 rows.
Market data read: 3185 rows.

Initializing Fast Network (MaxWait=2880 min)...
Building graph...
Graph built in 0.12s

Finding 5 paths per demand (Node Penalty=2)...
  Processed 500/3185... (278.7 req/s)
  Processed 1000/3185... (308.0 req/s)
  Processed 1500/3185... (283.2 req/s)
  Processed 2000/3185... (265.0 req/s)
  Processed 2500/3185... (254.1 req/s)
  Processed 3000/3185... (263.9 req/s)

Finished finding paths in 12.18 seconds.

--- Pathfinding Summary ---
Found results for 3185 ODTs.
Total paths found: 9254
Average paths per ODT: 2.91 (Target K=5)

Example for ODT 'AMS/LAX/1080':
  Path 1: Time=1850, Cap=47650.00, Flights=[24, 1306, 1685, 536]
  Path 2: Time=2110, Cap=46000.00, Flights=[25, 1317, 1116]
  Path 3: Time=1850, Cap=46000.00, Flights=[25, 1307, 1177, 536]
  Path 4: Time=2310, Cap=46000.00, Flights=[24, 1302, 347, 1313, 1846]
  Path 5: Time=2345, Cap=46000.00, Flights=[25, 1319, 163, 996]


In [92]:
# --- Simplified Network Class ---
class SimpleFlightNetwork:
    def __init__(self, schedule_df, demand_df, k_paths, min_connect_mins, max_wait_mins):
        print(f"\nInitializing Simplified Network (MaxWait={max_wait_mins} min)...")
        if schedule_df.empty or demand_df.empty: raise ValueError("Input data is empty.")

        self.k_paths = k_paths
        self.min_connect_mins = min_connect_mins
        self.max_wait_mins = max_wait_mins

        # --- Essential Lookups ---
        self.flight_data = schedule_df.set_index('flight_id').to_dict('index')
        self.demand_details = demand_df.set_index('ODT').to_dict('index')
        # Flights grouped by origin, containing sorted lists of IDs and departure times
        self.flights_by_origin = defaultdict(lambda: {'ids': [], 'dep_times': []})
        schedule_sorted = schedule_df.sort_values(by='dep_time')
        for idx, row in schedule_sorted.iterrows():
             key = row['ori']
             self.flights_by_origin[key]['ids'].append(row['flight_id'])
             self.flights_by_origin[key]['dep_times'].append(row['dep_time'])

        # --- Graph Representation ---
        self.graph = defaultdict(list)
        self._build_graph() # Underscore indicates intended private use

    def _add_edge(self, u, v, elapsed_time, flight_id=None, capacity=float('inf'), is_flight=False):
        """Adds edge to the graph."""
        if elapsed_time < 0: return
        cap = float(capacity) if math.isfinite(capacity) else float('inf')
        self.graph[u].append({
            'neighbor': v, 'weight': float(elapsed_time), 'flight_id': flight_id,
            'capacity': cap, 'is_flight': is_flight
        })

    def _find_first_flight_idx_after(self, dep_times_list, min_departure_time):
        """Finds index using bisect_left."""
        idx = bisect_left(dep_times_list, min_departure_time)
        return idx if idx < len(dep_times_list) else -1

    def _build_graph(self):
        """Builds pruned time-expanded graph with destination sinks."""
        print("Building simplified graph...")
        start_time = timer.time()
        # Node Types: ('source', odt), (flight_id, 'dep'), (flight_id, 'arr'), ('dest_sink', airport)

        # 1. Flight Edges (Dep -> Arr)
        for flight_id, flight in self.flight_data.items():
            self._add_edge((flight_id, 'dep'), (flight_id, 'arr'), flight['arr_time'] - flight['dep_time'],
                           flight_id, flight['cap_kg'], True)

        # 2. Connection (Arr -> Dep) and Sink (Arr -> Dest_Sink) Edges
        for flight1_id, flight1 in self.flight_data.items():
            arrival_node = (flight1_id, 'arr')
            arrival_time = flight1['arr_time']
            arrival_airport = flight1['des']

            # a) Destination Sink Edge (Arr -> Dest_Sink)
            self._add_edge(arrival_node, ('dest_sink', arrival_airport), 0)

            # b) Connection Edges (Arr -> Dep) with pruning
            origin_data = self.flights_by_origin.get(arrival_airport)
            if origin_data:
                min_dep_time = arrival_time + self.min_connect_mins
                start_idx = self._find_first_flight_idx_after(origin_data['dep_times'], min_dep_time)
                if start_idx != -1:
                    for i in range(start_idx, len(origin_data['ids'])):
                        wait_time = origin_data['dep_times'][i] - arrival_time
                        if wait_time <= self.max_wait_mins: # Pruning
                            self._add_edge(arrival_node, (origin_data['ids'][i], 'dep'), wait_time)
                        else: break

        # 3. Source Edges (Source -> Dep) with pruning
        for odt_id, demand in self.demand_details.items():
            source_node = ('source', odt_id)
            origin_data = self.flights_by_origin.get(demand['ori'])
            if origin_data:
                start_idx = self._find_first_flight_idx_after(origin_data['dep_times'], demand['time'])
                if start_idx != -1:
                    for i in range(start_idx, len(origin_data['ids'])):
                        initial_wait = origin_data['dep_times'][i] - demand['time']
                        if initial_wait <= self.max_wait_mins: # Pruning
                             self._add_edge(source_node, (origin_data['ids'][i], 'dep'), initial_wait)
                        else: break

        print(f"Graph building completed in {timer.time() - start_time:.2f} seconds.")

    def _a_star(self, start_node, end_node, edge_penalties=None):
        """A* Search (heuristic=0 -> Dijkstra). Returns (cost, path_list)."""
        global tie_breaker
        if edge_penalties is None: edge_penalties = {}
        heuristic = lambda u, v: 0.0 # Simple heuristic (Dijkstra)

        pq = [(heuristic(start_node, end_node), next(tie_breaker), 0.0, start_node, [{'node': start_node, 'edge_info': None}])] # f, tie, g, node, path
        visited_g_costs = defaultdict(lambda: float('inf'))
        visited_g_costs[start_node] = 0.0

        while pq:
            f, _, g, curr_node, path = heapq.heappop(pq)
            if g > visited_g_costs[curr_node]: continue
            if curr_node == end_node: return g, path # Return actual cost g

            if curr_node not in self.graph: continue
            for edge in self.graph[curr_node]:
                neighbor = edge['neighbor']
                edge_rep = (curr_node, neighbor, edge['flight_id'])
                penalty = edge_penalties.get(edge_rep, 1.0)
                penalized_cost = edge['weight'] * penalty
                new_g = g + penalized_cost

                if new_g < visited_g_costs[neighbor]:
                    visited_g_costs[neighbor] = new_g
                    h = heuristic(neighbor, end_node)
                    new_f = new_g + h
                    new_step = {'node': neighbor, 'edge_info': edge}
                    heapq.heappush(pq, (new_f, next(tie_breaker), new_g, neighbor, path + [new_step]))
        return float('inf'), None

    def _format_path(self, path_list, total_elapsed_time):
        """Formats path list into desired output."""
        if not path_list: return None
        flight_ids = []
        min_capacity = float('inf')
        for step in path_list[1:]:
            if isinstance(step['node'], tuple) and step['node'][0] == 'dest_sink': break
            edge = step['edge_info']
            if edge and edge['is_flight']:
                flight_ids.append(edge['flight_id'])
                min_capacity = min(min_capacity, edge['capacity'])
        min_cap = 0.0 if min_capacity == float('inf') else min_capacity
        return {'total_time': total_elapsed_time, 'flight_ids': flight_ids, 'min_capacity': min_cap}

    def find_k_paths(self, penalty_factor=1.01):
        """Finds K paths using A* and edge penalization heuristic."""
        print(f"\nFinding {self.k_paths} paths per demand (Heuristic Method)...")
        all_results = {}
        total_demands = len(self.demand_details)
        start_run_time = timer.time()

        for i, (odt_id, demand) in enumerate(self.demand_details.items()):
            if (i + 1) % 500 == 0: # Progress update
                elapsed = timer.time() - start_run_time
                rate = (i + 1) / elapsed if elapsed > 0 else 0
                print(f"  Processed {i+1}/{total_demands}... ({rate:.1f} req/s)")

            start_node = ('source', odt_id)
            target_node = ('dest_sink', demand['des'])
            found_paths = []
            edge_penalties = {}
            found_sequences = set()

            for k in range(self.k_paths):
                cost, path_list = self._a_star(start_node, target_node, edge_penalties)
                if path_list:
                    nodes_tuple = tuple(s['node'] for s in path_list)
                    if nodes_tuple not in found_sequences:
                        found_paths.append((cost, path_list))
                        found_sequences.add(nodes_tuple)
                        # Apply penalties
                        for step_idx in range(1, len(path_list)):
                            edge = path_list[step_idx]['edge_info']
                            if edge:
                                u, v, f_id = path_list[step_idx-1]['node'], path_list[step_idx]['node'], edge['flight_id']
                                edge_rep = (u, v, f_id)
                                edge_penalties[edge_rep] = edge_penalties.get(edge_rep, 1.0) * penalty_factor
                    else: break # Duplicate found, stop for this ODT
                else: break # No more paths found

            # Format results
            formatted = []
            found_paths.sort(key=lambda x: x[0])
            for cost, p_list in found_paths:
                 fmt = self._format_path(p_list, cost)
                 if fmt: formatted.append(fmt)
            all_results[odt_id] = formatted

        print(f"\nFinished finding paths in {timer.time() - start_run_time:.2f} seconds.")
        return all_results

# --- Main Execution Logic ---
print("Starting Simplified Flight Path Analysis...")
# 1. Read Data
schedule_data = read_capacity(CAPACITY_FILE, airport_substitutions)
demand_data = read_market(MARKET_FILE, airport_substitutions)

# 2. Initialize and Run

flight_network = SimpleFlightNetwork(
    schedule_data, demand_data, K_PATHS, MIN_CONNECT_MINS, MAX_CONNECT_WAIT_MINS
)
k_shortest_paths = flight_network.find_k_paths() # Renamed method slightly

# 3. Output Summary
print("\n--- Pathfinding Summary ---")
total_paths_found = sum(len(p) for p in k_shortest_paths.values())
num_odts = len(k_shortest_paths)
avg_paths = total_paths_found / num_odts if num_odts > 0 else 0
print(f"Found results for {num_odts} ODTs.")
print(f"Total paths found: {total_paths_found}")
print(f"Average paths per ODT: {avg_paths:.2f} (Target K={K_PATHS})")

# Example output for one ODT (optional)
first_odt = list(k_shortest_paths.keys())[0]
print(f"\nExample for ODT '{first_odt}':")
for i, path in enumerate(k_shortest_paths[first_odt]):
     print(f"  Path {i+1}: Time={path['total_time']:.0f}, Cap={path['min_capacity']:.2f}, Flights={path['flight_ids']}")

Starting Simplified Flight Path Analysis...
Capacity data read: 2057 rows.
Market data read: 3185 rows.

Initializing Simplified Network (MaxWait=2880 min)...
Building simplified graph...
Graph building completed in 0.08 seconds.

Finding 3 paths per demand (Heuristic Method)...
  Processed 500/3185... (597.7 req/s)
  Processed 1000/3185... (645.9 req/s)
  Processed 1500/3185... (559.7 req/s)
  Processed 2000/3185... (505.4 req/s)
  Processed 2500/3185... (479.7 req/s)
  Processed 3000/3185... (504.3 req/s)

Finished finding paths in 6.38 seconds.

--- Pathfinding Summary ---
Found results for 3185 ODTs.
Total paths found: 5226
Average paths per ODT: 1.64 (Target K=3)

Example for ODT 'AMS/LAX/1080':
  Path 1: Time=1850, Cap=47650.00, Flights=[24, 1306, 1685, 536]
  Path 2: Time=1853, Cap=46000.00, Flights=[25, 1307, 1177, 536]
  Path 3: Time=1869, Cap=47650.00, Flights=[24, 1307, 1177, 536]


In [83]:
# --- Network with Destination Sinks & Heuristic KSP ---
class FlightNetworkDestSinkHeuristic:
    def __init__(self, schedule_df, demand_df, k_paths, min_connect_mins, max_wait_mins):
        print(f"\nInitializing Network with Destination Sinks (MaxWait={max_wait_mins} min)...")
        if schedule_df.empty or demand_df.empty:
             raise ValueError("Schedule or Demand dataframe is empty.")

        self.k_paths = k_paths
        self.min_connect_mins = min_connect_mins
        self.max_wait_mins = max_wait_mins

        print("Precomputing lookups...")
        self.flight_data = schedule_df.set_index('flight_id').to_dict('index')
        self.demand_details = demand_df.set_index('ODT').to_dict('index')
        self.flights_by_origin = defaultdict(lambda: {'ids': [], 'dep_times': []})
        schedule_sorted = schedule_df.sort_values(by='dep_time')
        for idx, row in schedule_sorted.iterrows():
             key = row['ori']
             self.flights_by_origin[key]['ids'].append(row['flight_id'])
             self.flights_by_origin[key]['dep_times'].append(row['dep_time'])

        # Get list of unique destination airports in demands
        self.destination_airports = demand_df['des'].unique()

        # Graph Representation (Forward only needed for A*)
        self.graph = defaultdict(list)
        self.build_graph()

    def find_first_flight_idx_after(self, dep_times_list, min_departure_time):
        idx = bisect_left(dep_times_list, min_departure_time)
        return idx if idx < len(dep_times_list) else -1

    def _add_edge(self, u, v, elapsed_time, flight_id=None, capacity=float('inf'), is_flight=False):
        """Adds edge to the forward graph."""
        if elapsed_time < 0: return
        cap = float(capacity) if isinstance(capacity, (int, float)) and math.isfinite(capacity) else float('inf')
        self.graph[u].append({
            'neighbor': v, 'weight': float(elapsed_time), 'flight_id': flight_id,
            'capacity': cap, 'is_flight': is_flight
        })

    def build_graph(self):
        """Builds pruned time-expanded graph with destination sinks."""
        print("Building graph with destination sinks...")
        start_time = timer.time()

        # Node Types:
        # ('source', odt_str)
        # (flight_id_int, 'dep')
        # (flight_id_int, 'arr')
        # ('dest_sink', airport_str) # NEW SINK TYPE

        # 1. Flight Edges (Dep -> Arr)
        for flight_id, flight in self.flight_data.items():
            duration = flight['arr_time'] - flight['dep_time']
            self._add_edge((flight_id, 'dep'), (flight_id, 'arr'), duration, flight_id, flight['cap_kg'], True)

        # 2. Connection (Arr -> Dep) and Sink (Arr -> Dest_Sink) Edges
        connection_edge_count = 0
        sink_edge_count = 0
        for flight1_id, flight1 in self.flight_data.items():
            arrival_node = (flight1_id, 'arr')
            arrival_time = flight1['arr_time']
            arrival_airport = flight1['des']

            # a) Destination Sink Edge (Arr -> Dest_Sink, weight 0)
            # Only one edge per arriving flight to its destination airport sink
            dest_sink_node = ('dest_sink', arrival_airport)
            self._add_edge(arrival_node, dest_sink_node, 0)
            sink_edge_count += 1

            # b) Connection Edges (Arr -> Dep)
            origin_data = self.flights_by_origin.get(arrival_airport)
            if origin_data:
                min_dep_time = arrival_time + self.min_connect_mins
                start_idx = self.find_first_flight_idx_after(origin_data['dep_times'], min_dep_time)
                if start_idx != -1:
                    for i in range(start_idx, len(origin_data['ids'])):
                        flight2_id = origin_data['ids'][i]
                        flight2_dep_time = origin_data['dep_times'][i]
                        wait_time = flight2_dep_time - arrival_time
                        if wait_time <= self.max_wait_mins: # PRUNING
                            self._add_edge(arrival_node, (flight2_id, 'dep'), wait_time)
                            connection_edge_count += 1
                        else: break # Pruning based on wait time

        # 3. Source Edges (Source -> Dep)
        source_edge_count = 0
        for odt_id, demand in self.demand_details.items():
            source_node = ('source', odt_id)
            origin_airport = demand['ori']
            ready_time = demand['time']
            origin_data = self.flights_by_origin.get(origin_airport)
            if origin_data:
                start_idx = self.find_first_flight_idx_after(origin_data['dep_times'], ready_time)
                if start_idx != -1:
                    for i in range(start_idx, len(origin_data['ids'])):
                        flight_id = origin_data['ids'][i]
                        flight_dep_time = origin_data['dep_times'][i]
                        initial_wait = flight_dep_time - ready_time
                        if initial_wait <= self.max_wait_mins: # PRUNING
                            self._add_edge(source_node, (flight_id, 'dep'), initial_wait)
                            source_edge_count += 1
                        else: break # Pruning based on wait time

        print(f"Graph building completed in {timer.time() - start_time:.2f} seconds.")
        # Node count estimate: |Sources| + |Airports| + 2 * |Flights|
        est_nodes = len(self.demand_details) + len(self.destination_airports) + len(self.flight_data) * 2
        print(f"Graph nodes estimated around: {est_nodes}")
        print(f"Edge Counts: Flight={len(self.flight_data)}, Conn={connection_edge_count}, Sink={sink_edge_count}, Source={source_edge_count}")
        if sink_edge_count != len(self.flight_data):
             print("Warning: Sink edge count doesn't match flight count. Check logic.")


    def _a_star(self, start_node, end_node, heuristic=lambda u, v: 0, edge_penalties=None):
        """A* Search. Returns (cost, path_list)."""
        # (Same as previous version)
        global tie_breaker
        if edge_penalties is None: edge_penalties = {}
        g_cost_start = 0.0
        h_cost_start = heuristic(start_node, end_node)
        f_cost_start = g_cost_start + h_cost_start
        pq = [(f_cost_start, next(tie_breaker), g_cost_start, start_node, [{'node': start_node, 'edge_info': None}])]
        visited_g_costs = defaultdict(lambda: float('inf'))
        visited_g_costs[start_node] = g_cost_start

        while pq:
            f_cost, _, g_cost, current_node, path_list = heapq.heappop(pq)
            if g_cost > visited_g_costs[current_node]: continue
            if current_node == end_node: return g_cost, path_list

            if current_node not in self.graph: continue
            for edge in self.graph[current_node]:
                neighbor = edge['neighbor']
                edge_elapsed_time = edge['weight']
                flight_id = edge['flight_id']
                edge_rep = (current_node, neighbor, flight_id)
                penalty_factor = edge_penalties.get(edge_rep, 1.0)
                penalized_edge_time = edge_elapsed_time * penalty_factor
                new_g_cost = g_cost + penalized_edge_time

                if new_g_cost < visited_g_costs[neighbor]:
                    visited_g_costs[neighbor] = new_g_cost
                    h_cost_neighbor = heuristic(neighbor, end_node)
                    new_f_cost = new_g_cost + h_cost_neighbor
                    new_step = {'node': neighbor, 'edge_info': edge}
                    new_path = path_list + [new_step]
                    heapq.heappush(pq, (new_f_cost, next(tie_breaker), new_g_cost, neighbor, new_path))
        return float('inf'), None

    def _format_path(self, path_list, total_elapsed_time):
        """Formats path list into desired output."""
        # (Same as previous correct version)
        if not path_list or len(path_list) < 2: return None
        flight_ids = []
        min_capacity = float('inf')
        for step in path_list[1:]:
            edge = step['edge_info']
            # Stop processing if we hit the destination sink node
            if isinstance(step['node'], tuple) and step['node'][0] == 'dest_sink':
                break
            if edge and edge['is_flight']:
                flight_ids.append(edge['flight_id'])
                min_capacity = min(min_capacity, edge['capacity'])
        min_cap_final = 0.0 if min_capacity == float('inf') else min_capacity
        return {'total_time': total_elapsed_time, 'flight_ids': flight_ids, 'min_capacity': min_cap_final}

    def find_k_shortest_paths_heuristic(self, penalty_factor=1.01):
        """Finds K shortest paths using A* and edge penalization heuristic with destination sinks."""
        print(f"\nFinding {self.k_paths} shortest paths (Dest Sink Heuristic, Penalty={penalty_factor})...")
        all_results = {}
        total_demands = len(self.demand_details)
        start_run_time = timer.time()
        heuristic = lambda u, v: 0.0 # Simple Dijkstra-like heuristic

        for i, (odt_id, demand) in enumerate(self.demand_details.items()):
            if (i + 1) % 500 == 0:
                elapsed = timer.time() - start_run_time
                rate = (i + 1) / elapsed if elapsed > 0 else 0
                print(f"  Processed {i+1}/{total_demands} demands... Elapsed: {elapsed:.1f}s ({rate:.1f} req/s)")

            start_node = ('source', odt_id)
            # TARGET is now the destination airport sink
            target_node = ('dest_sink', demand['des'])

            found_paths_for_odt = [] # Stores (actual_cost, path_list)
            edge_penalties = {}
            found_path_node_sequences = set() # Track sequences to avoid duplicates

            for k in range(self.k_paths):
                actual_cost, path_list = self._a_star(start_node, target_node, heuristic, edge_penalties)

                if path_list:
                    nodes_tuple = tuple(s['node'] for s in path_list)
                    if nodes_tuple not in found_path_node_sequences:
                        found_paths_for_odt.append((actual_cost, path_list))
                        found_path_node_sequences.add(nodes_tuple) # Cache it

                        # Apply penalties for next iteration
                        for step_idx in range(1, len(path_list)):
                            edge = path_list[step_idx]['edge_info']
                            if edge:
                                u = path_list[step_idx-1]['node']
                                v = path_list[step_idx]['node']
                                f_id = edge['flight_id']
                                edge_rep = (u, v, f_id)
                                edge_penalties[edge_rep] = edge_penalties.get(edge_rep, 1.0) * penalty_factor
                    else:
                         # Found a duplicate path sequence, stop for this ODT
                         # print(f"  WARN: Duplicate path seq found for ODT {odt_id} at k={k+1}, stopping.")
                         break
                else:
                    break # No more paths found

            # Format results for this ODT
            formatted_paths = []
            found_paths_for_odt.sort(key=lambda x: x[0]) # Sort by actual cost
            for cost, path_list in found_paths_for_odt:
                 formatted = self._format_path(path_list, cost)
                 if formatted: formatted_paths.append(formatted)
            all_results[odt_id] = formatted_paths # Store results for this ODT

        print(f"\nFinished finding K shortest paths (Dest Sink Heuristic) in {timer.time() - start_run_time:.2f} seconds.")
        return all_results

# --- Main Execution Logic ---
print("Starting flight path analysis (Destination Sink + Heuristic KSP)...")
# 1. Read Data
schedule_data = read_capacity(CAPACITY_FILE, airport_substitutions)
demand_data = read_market(MARKET_FILE, airport_substitutions)

# 2. Initialize and Run only if data is valid
if not schedule_data.empty and not demand_data.empty:
    try:
        flight_network = FlightNetworkDestSinkHeuristic(
            schedule_data, demand_data, K_PATHS, MIN_CONNECT_MINS, MAX_CONNECT_WAIT_MINS
        )
        k_shortest_paths = flight_network.find_k_shortest_paths_heuristic()

        # 3. Output Sample Results
        print("\n--- K Shortest Paths Results (Sample - Dest Sink Heuristic) ---")
        output_count = 0
        total_paths_found_count = 0
        for odt, paths in k_shortest_paths.items():
            total_paths_found_count += len(paths)
            if output_count < 5:
                demand_info = flight_network.demand_details.get(odt, {})
                print(f"ODT: {odt} (Demand: {demand_info.get('ori','?')}-{demand_info.get('des','?')} @ {demand_info.get('time','?')})")
                if paths:
                    for i, path in enumerate(paths):
                        print(f"  Path {i+1}: Total Time={path['total_time']:.0f} min, Min Capacity={path['min_capacity']:.2f} kg, Flights={path['flight_ids']}")
                else:
                    print("  No paths found.")
                print("-" * 15)
            elif output_count == 5:
                 print("\n... (output truncated) ...")
            output_count += 1

        avg_paths = total_paths_found_count / len(k_shortest_paths) if k_shortest_paths else 0
        print(f"\nFound path results for {len(k_shortest_paths)} ODTs (Average {avg_paths:.2f} paths/ODT).")
        if avg_paths < K_PATHS:
            print(f"NOTE: Average paths found is less than K={K_PATHS}. This can happen with pruning or the heuristic KSP method.")

    except ValueError as e:
        print(f"\nError during processing: {e}")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")
        import traceback
        traceback.print_exc()
else:
    print("\nError: Could not read valid schedule or demand data.")

Starting flight path analysis (Destination Sink + Heuristic KSP)...
Capacity data read: 2057 rows.
Market data read: 3185 rows.

Initializing Network with Destination Sinks (MaxWait=2880 min)...
Precomputing lookups...
Building graph with destination sinks...
Graph building completed in 0.12 seconds.
Graph nodes estimated around: 7376
Edge Counts: Flight=2057, Conn=52153, Sink=2057, Source=52686

Finding 3 shortest paths (Dest Sink Heuristic, Penalty=1.01)...
  Processed 500/3185 demands... Elapsed: 0.8s (596.0 req/s)
  Processed 1000/3185 demands... Elapsed: 1.6s (629.2 req/s)
  Processed 1500/3185 demands... Elapsed: 2.8s (540.5 req/s)
  Processed 2000/3185 demands... Elapsed: 4.1s (487.2 req/s)
  Processed 2500/3185 demands... Elapsed: 5.5s (457.8 req/s)
  Processed 3000/3185 demands... Elapsed: 6.2s (485.8 req/s)

Finished finding K shortest paths (Dest Sink Heuristic) in 6.61 seconds.

--- K Shortest Paths Results (Sample - Dest Sink Heuristic) ---
ODT: AMS/LAX/1080 (Demand: AMS-L

In [63]:
import pandas as pd
import heapq
from collections import defaultdict
import itertools
import math
import time as timer
from bisect import bisect_left # Can potentially optimize connection finding slightly

# --- Global Tie-breaker for Dijkstra Heapq ---
tie_breaker = itertools.count()

# --- Efficient Time-Expanded Flight Network ---
class FlightNetworkEfficient:
    def __init__(self, schedule_df, demand_df, k_paths, min_connect_mins):
        print("\nInitializing Efficient Time-Expanded Flight Network...")
        if schedule_df.empty or demand_df.empty:
             raise ValueError("Schedule or Demand dataframe is empty.")

        self.k_paths = k_paths
        self.min_connect_mins = min_connect_mins

        # --- Efficient Lookups ---
        print("Precomputing lookups...")
        self.flight_data = schedule_df.set_index('flight_id').to_dict('index')
        self.demand_details = demand_df.set_index('ODT').to_dict('index')

        # Group flights by origin AND store sorted departure times list for bisect
        self.flights_by_origin = defaultdict(lambda: {'ids': [], 'dep_times': []})
        # Sort once globally by departure time
        schedule_sorted = schedule_df.sort_values(by='dep_time')
        for idx, row in schedule_sorted.iterrows():
             key = row['ori']
             self.flights_by_origin[key]['ids'].append(row['flight_id'])
             self.flights_by_origin[key]['dep_times'].append(row['dep_time'])

        # Pre-group demands by destination airport for faster sink edge lookup
        self.demands_by_dest = defaultdict(list)
        for odt_id, demand_info in self.demand_details.items():
             self.demands_by_dest[demand_info['des']].append(odt_id)


        # --- Graph Representation: Adjacency List ---
        self.graph = defaultdict(list)
        self.build_graph()

    def _add_edge(self, u, v, elapsed_time, flight_id=None, capacity=float('inf'), is_flight=False):
        """Adds edge with elapsed_time as weight."""
        if elapsed_time < 0: return

        cap = float(capacity) if isinstance(capacity, (int, float)) and math.isfinite(capacity) else float('inf')
        self.graph[u].append({
            'neighbor': v, 'weight': float(elapsed_time), 'flight_id': flight_id,
            'capacity': cap, 'is_flight': is_flight
        })

    def find_first_flight_idx_after(self, dep_times_list, min_departure_time):
        """Finds index of first flight >= min_departure_time using bisect_left."""
        idx = bisect_left(dep_times_list, min_departure_time)
        return idx if idx < len(dep_times_list) else -1

    def build_graph(self):
        """Builds the time-expanded graph efficiently."""
        print("Building time-expanded graph...")
        start_time = timer.time()

        # Node Types: ('source', odt), ('sink', odt), (flight_id, 'dep'), (flight_id, 'arr')

        # 1. Flight Edges (Dep -> Arr)
        for flight_id, flight in self.flight_data.items():
            duration = flight['arr_time'] - flight['dep_time']
            self._add_edge((flight_id, 'dep'), (flight_id, 'arr'), duration, flight_id, flight['cap_kg'], True)

        # 2. Connection (Arr -> Dep) and Sink (Arr -> Sink) Edges
        connection_edge_count = 0
        sink_edge_count = 0
        for flight1_id, flight1 in self.flight_data.items():
            arrival_node = (flight1_id, 'arr')
            arrival_time = flight1['arr_time']
            arrival_airport = flight1['des']

            # a) Sink Edges (Efficiently using precomputed demands_by_dest)
            for odt_id in self.demands_by_dest.get(arrival_airport, []):
                 self._add_edge(arrival_node, ('sink', odt_id), 0)
                 sink_edge_count += 1

            # b) Connection Edges (Using bisect for potential speedup)
            origin_data = self.flights_by_origin.get(arrival_airport)
            if origin_data:
                min_dep_time = arrival_time + self.min_connect_mins
                # Find index of first potentially valid flight
                start_idx = self.find_first_flight_idx_after(origin_data['dep_times'], min_dep_time)

                if start_idx != -1:
                    # Add edges for all valid flights from this index onwards
                    for i in range(start_idx, len(origin_data['ids'])):
                        flight2_id = origin_data['ids'][i]
                        flight2_dep_time = origin_data['dep_times'][i] # Already available
                        wait_time = flight2_dep_time - arrival_time
                        self._add_edge(arrival_node, (flight2_id, 'dep'), wait_time)
                        connection_edge_count += 1

        # 3. Source Edges (Source -> Dep)
        source_edge_count = 0
        for odt_id, demand in self.demand_details.items():
            source_node = ('source', odt_id)
            origin_airport = demand['ori']
            ready_time = demand['time']

            origin_data = self.flights_by_origin.get(origin_airport)
            if origin_data:
                # Find index of first potentially valid flight
                start_idx = self.find_first_flight_idx_after(origin_data['dep_times'], ready_time)
                if start_idx != -1:
                    # Add edges for all valid flights from this index onwards
                    for i in range(start_idx, len(origin_data['ids'])):
                        flight_id = origin_data['ids'][i]
                        flight_dep_time = origin_data['dep_times'][i]
                        initial_wait = flight_dep_time - ready_time
                        self._add_edge(source_node, (flight_id, 'dep'), initial_wait)
                        source_edge_count += 1

        print(f"Graph building completed in {timer.time() - start_time:.2f} seconds.")
        est_nodes = len(self.demand_details) * 2 + len(self.flight_data) * 2
        print(f"Graph nodes estimated around: {est_nodes}")
        print(f"Edge Counts: Flight={len(self.flight_data)}, Connection={connection_edge_count}, Sink={sink_edge_count}, Source={source_edge_count}")


    def _dijkstra(self, start_node, end_node, forbidden_edges=None, forbidden_nodes=None):
        """Standard Dijkstra minimizing total elapsed time."""
        global tie_breaker
        if forbidden_edges is None: forbidden_edges = set()
        if forbidden_nodes is None: forbidden_nodes = set()

        pq = [(0.0, next(tie_breaker), start_node, [{'node': start_node, 'edge_info': None}])]
        visited_costs = defaultdict(lambda: float('inf'))
        visited_costs[start_node] = 0.0

        while pq:
            cost, _, current_node, path_list = heapq.heappop(pq)

            if cost > visited_costs[current_node]: continue
            if current_node == end_node: return cost, path_list

            if current_node not in self.graph: continue

            for edge in self.graph[current_node]:
                neighbor = edge['neighbor']
                edge_elapsed_time = edge['weight']

                if neighbor in forbidden_nodes: continue
                edge_rep = (current_node, neighbor, edge['flight_id'])
                if edge_rep in forbidden_edges: continue

                new_cost = cost + edge_elapsed_time
                if new_cost < visited_costs[neighbor]:
                    visited_costs[neighbor] = new_cost
                    new_step = {'node': neighbor, 'edge_info': edge}
                    new_path = path_list + [new_step]
                    heapq.heappush(pq, (new_cost, next(tie_breaker), neighbor, new_path))

        return float('inf'), None


    def _format_path(self, path_list, total_elapsed_time):
        """Formats path list into desired output."""
        if not path_list or len(path_list) < 2: return None
        flight_ids = []
        min_capacity = float('inf')
        for step in path_list[1:]:
            edge = step['edge_info']
            if edge and edge['is_flight']:
                flight_ids.append(edge['flight_id'])
                min_capacity = min(min_capacity, edge['capacity'])
        min_cap_final = 0.0 if min_capacity == float('inf') else min_capacity
        return {'total_time': total_elapsed_time, 'flight_ids': flight_ids, 'min_capacity': min_cap_final}


    def find_k_shortest_paths(self):
        """Finds K shortest paths using Yen's."""
        print(f"\nFinding {self.k_paths} shortest paths...")
        all_results = {}
        total_demands = len(self.demand_details)
        start_run_time = timer.time()

        for i, (odt_id, demand) in enumerate(self.demand_details.items()):
            if (i + 1) % 50 == 0: # Progress update every 50 demands
                elapsed = timer.time() - start_run_time
                rate = (i + 1) / elapsed if elapsed > 0 else 0
                print(f"  Processed {i+1}/{total_demands} demands... Elapsed: {elapsed:.1f}s ({rate:.1f} req/s)")

            start_node = ('source', odt_id)
            end_node = ('sink', odt_id)

            A = [] # Stores final paths: (total_time_cost, path_list)
            B = [] # Candidate heap: (total_time_cost, tie_id, path_list)
            found_path_nodes = set() # Cache node sequences of paths added to A or B

            # 1. Find first shortest path
            cost1, path1 = self._dijkstra(start_node, end_node)
            if path1:
                A.append((cost1, path1))
                found_path_nodes.add(tuple(s['node'] for s in path1))
            else:
                all_results[odt_id] = []
                continue

            # 2. Find paths k = 2 to K
            for k in range(1, self.k_paths):
                if k > len(A): break
                prev_cost, prev_path = A[k-1]

                # Iterate through nodes of path k-1 to find spur points
                for i in range(len(prev_path) - 1):
                    spur_node = prev_path[i]['node']
                    root_path = prev_path[:i+1]
                    root_nodes_tuple = tuple(step['node'] for step in root_path)

                    forbidden_nodes = {step['node'] for step in root_path[:-1]}
                    forbidden_edges = set()

                    # Forbid edges leaving spur node used by paths A[0]..A[k-1] sharing the same root
                    for cost_j, path_j in A:
                        if len(path_j) > i + 1 and tuple(step['node'] for step in path_j[:i+1]) == root_nodes_tuple:
                            edge_info_j = path_j[i+1]['edge_info']
                            if edge_info_j:
                                edge_rep = (spur_node, edge_info_j['neighbor'], edge_info_j['flight_id'])
                                forbidden_edges.add(edge_rep)

                    # Rerun Dijkstra from START with accumulated forbids
                    spur_cost, spur_path_list = self._dijkstra(start_node, end_node, forbidden_edges, forbidden_nodes)

                    if spur_path_list:
                        spur_nodes_tuple = tuple(s['node'] for s in spur_path_list)
                        # Add to candidate heap B only if it's a genuinely new path sequence
                        if spur_nodes_tuple not in found_path_nodes:
                             heapq.heappush(B, (spur_cost, next(tie_breaker), spur_path_list))
                             found_path_nodes.add(spur_nodes_tuple) # Add to cache

                # Select the best path from B (lowest cost) that isn't already in A
                found_next = False
                while B:
                    potential_cost, _, potential_path = heapq.heappop(B)
                    # Check if this exact path was already added to A in a previous K iteration
                    # (Uniqueness based on cost AND sequence is implicitly handled by found_path_nodes)
                    # If we reach here, it's a valid candidate for the k-th path
                    A.append((potential_cost, potential_path))
                    found_next = True
                    break

                if not found_next: break

            # Format results for ODT
            formatted_paths = []
            A.sort(key=lambda x: x[0]) # Ensure sorted output by total time
            for cost, path_list in A:
                 formatted = self._format_path(path_list, cost)
                 if formatted: formatted_paths.append(formatted)
            all_results[odt_id] = formatted_paths[:self.k_paths]

        print(f"\nFinished finding K shortest paths in {timer.time() - start_run_time:.2f} seconds.")
        return all_results


# --- Main Execution Logic ---
print("Starting flight path analysis (Efficient Time-Expanded Model)...")
# 1. Read Data
schedule_data = read_capacity(CAPACITY_FILE, airport_substitutions)
demand_data = read_market(MARKET_FILE, airport_substitutions)

# 2. Initialize and Run only if data is valid
if not schedule_data.empty and not demand_data.empty:
    try:
        flight_network = FlightNetworkEfficient(schedule_data, demand_data, K_PATHS, MIN_CONNECT_MINS)
        k_shortest_paths = flight_network.find_k_shortest_paths()

        # 3. Output Sample Results
        print("\n--- K Shortest Paths Results (Sample) ---")
        output_count = 0
        total_paths_found_count = 0
        for odt, paths in k_shortest_paths.items():
            total_paths_found_count += len(paths)
            if output_count < 5:
                demand_info = flight_network.demand_details.get(odt, {})
                print(f"ODT: {odt} (Demand: {demand_info.get('ori','?')}-{demand_info.get('des','?')} @ {demand_info.get('time','?')})")
                if paths:
                    for i, path in enumerate(paths):
                        print(f"  Path {i+1}: Total Time={path['total_time']:.0f} min, Min Capacity={path['min_capacity']:.2f} kg, Flights={path['flight_ids']}")
                else:
                    print("  No paths found.")
                print("-" * 15)
            elif output_count == 5:
                 print("\n... (output truncated) ...")
            output_count += 1

        avg_paths = total_paths_found_count / len(k_shortest_paths) if k_shortest_paths else 0
        print(f"\nFound path results for {len(k_shortest_paths)} ODTs (Average {avg_paths:.2f} paths/ODT).")

    except ValueError as e:
        print(f"\nError during processing: {e}")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")
        import traceback
        traceback.print_exc()
else:
    print("\nError: Could not read valid schedule or demand data.")

Starting flight path analysis (Efficient Time-Expanded Model)...
Capacity data read: 2057 rows.
Market data read: 3185 rows.

Initializing Efficient Time-Expanded Flight Network...
Precomputing lookups...
Building time-expanded graph...
Graph building completed in 0.32 seconds.
Graph nodes estimated around: 10484
Edge Counts: Flight=2057, Connection=96571, Sink=121331, Source=99076

Finding 3 shortest paths...
  Processed 50/3185 demands... Elapsed: 1.3s (39.2 req/s)
  Processed 100/3185 demands... Elapsed: 2.4s (41.2 req/s)
  Processed 150/3185 demands... Elapsed: 3.6s (41.9 req/s)
  Processed 200/3185 demands... Elapsed: 5.4s (36.8 req/s)
  Processed 250/3185 demands... Elapsed: 6.9s (36.1 req/s)
  Processed 300/3185 demands... Elapsed: 8.2s (36.5 req/s)
  Processed 350/3185 demands... Elapsed: 10.0s (34.8 req/s)
  Processed 400/3185 demands... Elapsed: 11.5s (34.9 req/s)
  Processed 450/3185 demands... Elapsed: 14.0s (32.1 req/s)
  Processed 500/3185 demands... Elapsed: 15.8s (31.6 

In [49]:
# --- Flight-Vertex Network Class (Optimized within User's Model) ---
class FlightVertexNetworkOptimized:
    def __init__(self, schedule_df, demand_df, k_paths, min_connect_mins):
        print("\nInitializing Optimized Flight-Vertex Network...")
        if schedule_df.empty or demand_df.empty:
            raise ValueError("Schedule or Demand dataframe is empty.")

        self.k_paths = k_paths
        self.min_connect_mins = min_connect_mins

        # --- Essential Lookups ---
        print("Precomputing lookups...")
        self.flight_data = schedule_df.set_index('flight_id').to_dict('index')
        self.demand_details = demand_df.set_index('ODT').to_dict('index')

        # Store departure times separately for quick access during bisect
        self.flight_dep_times = schedule_df.set_index('flight_id')['dep_time'].to_dict()

        # Group flights by (ori, des) and sort by departure time
        # Store both flight_ids and corresponding dep_times for efficient bisect
        self.flights_by_od = defaultdict(lambda: {'ids': [], 'dep_times': []})
        schedule_sorted_od = schedule_df.sort_values(by=['ori', 'des', 'dep_time'])
        for idx, row in schedule_sorted_od.iterrows():
            key = (row['ori'], row['des'])
            self.flights_by_od[key]['ids'].append(row['flight_id'])
            self.flights_by_od[key]['dep_times'].append(row['dep_time'])

        # Group flights by origin, sorted by departure (needed for connections)
        # Store both flight_ids and dep_times
        self.flights_by_origin = defaultdict(lambda: {'ids': [], 'dep_times': []})
        schedule_sorted_ori = schedule_df.sort_values(by=['ori', 'dep_time'])
        for idx, row in schedule_sorted_ori.iterrows():
             key = row['ori']
             self.flights_by_origin[key]['ids'].append(row['flight_id'])
             self.flights_by_origin[key]['dep_times'].append(row['dep_time'])

        # --- Graph Representation (Adjacency List) ---
        # Node -> list of outgoing edges
        # Node types: ('source', odt_str), flight_id_int, ('sink', odt_str)
        # Edge dict: {'neighbor': node, 'cost': float (user_defined!),
        #             'capacity': float, 'is_waiting_edge': bool}
        # Note: We don't strictly need 'arr_time' on the edge if Dijkstra uses the cost correctly
        self.graph = defaultdict(list)
        self.build_graph()

    def _add_edge(self, u, v, user_cost, capacity=None, is_waiting_edge=False):
        """Adds edge with user-defined cost."""
        if u is None or v is None or user_cost < 0: return

        cap = float(capacity) if isinstance(capacity, (int, float)) and math.isfinite(capacity) else float('inf')

        self.graph[u].append({
            'neighbor': v,
            'cost': float(user_cost), # The cost metric requested by the user
            'capacity': cap,
            'is_waiting_edge': is_waiting_edge
        })

    def find_first_flight_idx_after(self, dep_times_list, min_departure_time):
        """Finds the index of the first flight departing at or after min_departure_time using bisect."""
        # bisect_left finds the insertion point for min_departure_time in the sorted list
        # This insertion point is the index of the first element >= min_departure_time
        idx = bisect_left(dep_times_list, min_departure_time)
        if idx < len(dep_times_list):
            return idx # Found a valid flight index
        else:
            return -1 # No flight found after the specified time

    def build_graph(self):
        """Builds the graph based on user's flight-vertex model rules using optimizations."""
        print("Building optimized flight-vertex graph...")
        start_time = timer.time()

        # 1. Source to Flight Edges (Optimized)
        print("1. Adding Source -> Flight edges...")
        source_edge_count = 0
        # Get unique origins from demands for faster iteration
        demand_origins = self.demand_details.keys()

        for odt_id, demand in self.demand_details.items():
            source_node = ('source', odt_id)
            demand_ori = demand['ori']
            demand_time = demand['time']
            processed_first_ods = set() # Track ODs processed for *this specific source*

            # Iterate relevant OD pairs starting from demand_ori more efficiently
            # Get all OD pairs starting from demand_ori
            relevant_od_pairs = [od for od in self.flights_by_od.keys() if od[0] == demand_ori]

            for od_pair in relevant_od_pairs:
                 if od_pair not in processed_first_ods:
                     od_data = self.flights_by_od[od_pair]
                     # Find index of first flight using bisect_left
                     idx = self.find_first_flight_idx_after(od_data['dep_times'], demand_time)

                     if idx != -1: # Found a valid flight
                         first_flight_id = od_data['ids'][idx]
                         flight = self.flight_data[first_flight_id]
                         # User Cost: dep_time - ready_time
                         user_cost = flight['dep_time'] - demand_time
                         self._add_edge(source_node, first_flight_id, user_cost, capacity=flight['cap_kg'])
                         processed_first_ods.add(od_pair)
                         source_edge_count += 1
        print(f"   Added {source_edge_count} source edges.")


        # 2. Flight to Flight Edges (Connections and Waiting - Optimized)
        print("2. Adding Flight -> Flight edges...")
        connection_edge_count = 0
        waiting_edge_count = 0
        for flight_id_A, flight_A in self.flight_data.items():
            arrival_airport_A = flight_A['des']
            arrival_time_A = flight_A['arr_time']
            departure_time_A = flight_A['dep_time']
            ori_A, des_A = flight_A['ori'], flight_A['des']

            # a) Connection Edges (A -> B)
            processed_first_conn_ods = set()
            origin_data = self.flights_by_origin.get(arrival_airport_A)
            if origin_data: # Check if any flights depart from arrival airport
                # Find first potential connecting flight using bisect
                min_dep_time_B = arrival_time_A + self.min_connect_mins
                start_idx_B = self.find_first_flight_idx_after(origin_data['dep_times'], min_dep_time_B)

                if start_idx_B != -1:
                    # Check flights from start_idx_B onwards to find first unique ODs
                    for i in range(start_idx_B, len(origin_data['ids'])):
                        flight_id_B = origin_data['ids'][i]
                        flight_B = self.flight_data[flight_id_B]
                        od_pair_B = (flight_B['ori'], flight_B['des'])

                        if od_pair_B not in processed_first_conn_ods:
                            # User Cost: B.dep_time - A.dep_time
                            user_cost = flight_B['dep_time'] - departure_time_A
                            self._add_edge(flight_id_A, flight_id_B, user_cost, capacity=flight_B['cap_kg'])
                            processed_first_conn_ods.add(od_pair_B)
                            connection_edge_count += 1
                            # Optimization: Could potentially break if we only need *one* connection total?
                            # User rule says "first unique OD", so we continue checking others.

            # b) Waiting Edges (A -> C, same OD)
            od_data = self.flights_by_od.get((ori_A, des_A))
            if od_data:
                 # Find the index of flight_A efficiently using bisect_left on dep_times
                 # This gives the insertion point, which might be the index if value exists
                 # A direct list.index() might be faster if lists are typically short
                 try:
                     # Use list.index() - likely faster for non-huge lists than recreating times
                     idx_A = od_data['ids'].index(flight_id_A)
                 except ValueError:
                     idx_A = -1 # Should not happen

                 if idx_A != -1 and idx_A + 1 < len(od_data['ids']):
                    flight_id_C = od_data['ids'][idx_A + 1]
                    flight_C = self.flight_data[flight_id_C]
                    # User Cost: C.dep_time - A.dep_time
                    user_cost = flight_C['dep_time'] - departure_time_A
                    self._add_edge(flight_id_A, flight_id_C, user_cost, capacity=flight_C['cap_kg'], is_waiting_edge=True)
                    waiting_edge_count += 1

        print(f"   Added {connection_edge_count} connection edges.")
        print(f"   Added {waiting_edge_count} 'waiting' edges.")

        # 3. Flight to Sink Edges (No change in logic needed, still potentially many)
        print("3. Adding Flight -> Sink edges...")
        sink_edge_count = 0
        # Pre-group demands by destination for efficiency
        demands_by_dest = defaultdict(list)
        for odt_id, demand in self.demand_details.items():
             demands_by_dest[demand['des']].append(odt_id)

        for flight_id, flight in self.flight_data.items():
            dest_airport = flight['des']
            # Connect to ALL demand sinks that require this destination
            for odt_id in demands_by_dest.get(dest_airport, []):
                 sink_node = ('sink', odt_id)
                 # User Cost: Duration of flight F
                 user_cost = flight['arr_time'] - flight['dep_time']
                 # No extra attributes needed on edge for basic Dijkstra with this cost model
                 self._add_edge(flight_id, sink_node, user_cost)
                 sink_edge_count += 1
        print(f"   Added {sink_edge_count} flight-to-sink edges.")


        print(f"Graph building completed in {timer.time() - start_time:.2f} seconds.")
        est_nodes = len(self.demand_details) + len(self.flight_data) + len(self.demand_details)
        print(f"Graph nodes estimated around: {est_nodes}")


    def _dijkstra(self, start_node, end_node, forbidden_edges=None, forbidden_nodes=None):
        """
        Standard Dijkstra minimizing the sum of user-defined edge costs.
        Returns (min_user_cost, path_list)
        """
        global tie_breaker
        if forbidden_edges is None: forbidden_edges = set()
        if forbidden_nodes is None: forbidden_nodes = set()

        # PQ: (user_cost, tie_id, current_node, path_list)
        # path_list: [{'node': node, 'edge_info': edge_dict_or_None}]
        pq = [(0.0, next(tie_breaker), start_node, [{'node': start_node, 'edge_info': None}])]
        visited_user_costs = defaultdict(lambda: float('inf'))
        visited_user_costs[start_node] = 0.0

        while pq:
            user_cost, _, current_node, path_list = heapq.heappop(pq)

            if user_cost > visited_user_costs[current_node]: continue
            if current_node == end_node: return user_cost, path_list # Found shortest path based on user_cost

            if current_node not in self.graph: continue

            for edge in self.graph[current_node]:
                neighbor_node = edge['neighbor']
                edge_user_cost = edge['cost']

                # --- Forbidden checks ---
                if neighbor_node in forbidden_nodes: continue
                neighbor_flight_id = neighbor_node if isinstance(neighbor_node, int) else None
                edge_rep = (current_node, neighbor_node, neighbor_flight_id)
                if edge_rep in forbidden_edges: continue
                # --- End Forbidden ---

                new_user_cost = user_cost + edge_user_cost
                if new_user_cost < visited_user_costs[neighbor_node]:
                    visited_user_costs[neighbor_node] = new_user_cost
                    new_step = {'node': neighbor_node, 'edge_info': edge}
                    new_path = path_list + [new_step]
                    heapq.heappush(pq, (new_user_cost, next(tie_breaker), neighbor_node, new_path))

        return float('inf'), None # No path found


    def _format_path(self, path_list, total_user_cost, demand_ready_time):
        """Formats path, extracting flights, capacity. Total time IS the user_cost sum."""
        if not path_list or len(path_list) < 2: return None

        flight_ids = []
        min_capacity = float('inf')
        final_arrival_time = demand_ready_time # Initialize for duration calculation

        for step in path_list[1:]: # Skip source
            node = step['node']
            edge = step['edge_info']

            # Identify "taken" flights (not waiting edges leading to them)
            # A flight is "taken" if the edge leading to it is NOT a waiting edge.
            # Or if it's the first flight from the source.
            # Or if it's the target of a connection edge.
            # Exception: If the *last* edge is flight->sink, that flight was taken.
            is_taken_flight = False
            prev_node = path_list[path_list.index(step)-1]['node']

            if isinstance(node, int): # Current step is a flight node
                if edge and not edge['is_waiting_edge']:
                     is_taken_flight = True
                # Update final_arrival_time based on the flight data IF it's the last flight
                # We need to know if the *next* node is the sink
                current_index = path_list.index(step)
                if current_index + 1 < len(path_list):
                     next_node_info = path_list[current_index+1]
                     if isinstance(next_node_info['node'], tuple) and next_node_info['node'][0] == 'sink':
                          final_arrival_time = self.flight_data[node]['arr_time']
                else: # This flight node is the last in path list (shouldn't happen if sink exists)
                     final_arrival_time = self.flight_data[node]['arr_time']


            if is_taken_flight and edge:
                 flight_ids.append(node)
                 min_capacity = min(min_capacity, edge['capacity'])

        min_cap_final = 0.0 if min_capacity == float('inf') else min_capacity

        # total_user_cost *is* the total elapsed time in this model
        total_elapsed_time = total_user_cost if total_user_cost != float('inf') else float('inf')
        # Double check calculation using final arrival if available
        if final_arrival_time > demand_ready_time:
            calculated_duration = final_arrival_time - demand_ready_time
            # If the sum didn't match (e.g., due to floating point), use calculated?
            # For now, trust the Dijkstra sum matches the algebraic simplification.
            # print(f"Debug: Dijkstra Cost = {total_user_cost}, Calc Duration = {calculated_duration}")
        else:
             # Handle cases where path doesn't end properly at a flight before sink?
             pass


        return {
            'total_time': total_elapsed_time, # Actual elapsed duration is the sum of user costs
            'flight_ids': flight_ids,    # Flights considered "taken"
            'min_capacity': min_cap_final
        }


    def find_k_shortest_paths(self):
        """Finds K shortest paths using Yen's, ranking by the sum of user_costs (== total time)."""
        print(f"\nFinding {self.k_paths} shortest paths (Flight-Vertex Model, ranked by user_cost)...")
        print("NOTE: Pathfinding uses user-defined costs. Ranking is based on the sum, which equals total duration.")
        print("      Graph pruning (first unique OD) may exclude globally optimal paths.")

        all_results = {}
        total_demands = len(self.demand_details)
        start_run_time = timer.time()

        for i, (odt_id, demand) in enumerate(self.demand_details.items()):
            if (i + 1) % 100 == 0:
                elapsed = timer.time() - start_run_time
                print(f"  Processed {i+1}/{total_demands} demands... Elapsed: {elapsed:.2f}s")

            start_node = ('source', odt_id)
            end_node = ('sink', odt_id)
            demand_ready_time = demand['time']

            A = [] # Stores final paths: (user_cost, path_list) - Sort by user_cost
            B = [] # Candidate heap: (user_cost, tie_id, path_list) - Heap by user_cost

            # 1. Find first shortest path based on user_cost
            cost1, path1 = self._dijkstra(start_node, end_node)
            if path1:
                A.append((cost1, path1))
            else:
                all_results[odt_id] = []
                continue

            found_path_nodes = {tuple(s['node'] for s in path1)} if path1 else set()

            # 2. Find paths k = 2 to K
            for k in range(1, self.k_paths):
                if k > len(A): break
                prev_cost, prev_path = A[k-1] # Path k-1 (0-indexed)

                # Iterate through nodes of path k-1 to find spur points
                for i in range(len(prev_path) - 1): # Spur node cannot be sink
                    spur_node = prev_path[i]['node']
                    root_path = prev_path[:i+1]

                    forbidden_nodes = {step['node'] for step in root_path[:-1]}
                    forbidden_edges = set()

                    # Forbid edges leaving spur node used by paths A[0]..A[k-1] sharing the same root
                    root_nodes_tuple = tuple(step['node'] for step in root_path)
                    for cost_j, path_j in A:
                        if len(path_j) > i + 1:
                            if tuple(step['node'] for step in path_j[:i+1]) == root_nodes_tuple:
                                edge_info_j = path_j[i+1]['edge_info']
                                if edge_info_j:
                                     neighbor_j = edge_info_j['neighbor']
                                     neighbor_flight_id_j = neighbor_j if isinstance(neighbor_j, int) else None
                                     edge_rep = (spur_node, neighbor_j, neighbor_flight_id_j)
                                     forbidden_edges.add(edge_rep)

                    # --- Calculate spur path ---
                    # Rerun Dijkstra from START with accumulated forbids
                    spur_cost, spur_path_list = self._dijkstra(start_node, end_node, forbidden_edges, forbidden_nodes)

                    if spur_path_list:
                        spur_nodes_tuple = tuple(s['node'] for s in spur_path_list)
                        # Add to candidate heap B only if it's a new path sequence
                        if spur_nodes_tuple not in found_path_nodes:
                             heapq.heappush(B, (spur_cost, next(tie_breaker), spur_path_list))
                             found_path_nodes.add(spur_nodes_tuple)

                # Select the best path from B (lowest user_cost) not already in A
                found_next = False
                while B:
                    potential_cost, _, potential_path = heapq.heappop(B)
                    # Path uniqueness already handled by found_path_nodes cache when adding to B
                    A.append((potential_cost, potential_path))
                    found_next = True
                    break # Found path k

                if not found_next: break

            # Format results for ODT
            formatted_paths = []
            A.sort(key=lambda x: x[0]) # Ensure sorted output by user_cost (== total time)
            for cost, path_list in A:
                 # Pass demand_ready_time for reference, although cost is the duration
                 formatted = self._format_path(path_list, cost, demand_ready_time)
                 if formatted: formatted_paths.append(formatted)
            all_results[odt_id] = formatted_paths[:self.k_paths] # Ensure only K paths

        print(f"\nFinished finding K shortest paths in {timer.time() - start_run_time:.2f} seconds.")
        return all_results


# --- Main Execution Logic ---
print("Starting flight path analysis (Optimized Flight-Vertex Model)...")
# 1. Read Data
schedule_data = read_capacity(CAPACITY_FILE, airport_substitutions)
demand_data = read_market(MARKET_FILE, airport_substitutions)

# 2. Initialize and Run only if data is valid
if not schedule_data.empty and not demand_data.empty:
    try:
        # Use the Optimized Flight-Vertex class
        flight_network = FlightVertexNetworkOptimized(schedule_data, demand_data, K_PATHS, MIN_CONNECT_MINS)
        k_shortest_paths = flight_network.find_k_shortest_paths()

        # 3. Output Sample Results
        print("\n--- K Shortest Paths Results (Sample - Ranked by Total Time) ---")
        output_count = 0
        for odt, paths in k_shortest_paths.items():
            if output_count < 5:
                demand_info = flight_network.demand_details.get(odt, {})
                print(f"ODT: {odt} (Demand: {demand_info.get('ori','?')}-{demand_info.get('des','?')} @ {demand_info.get('time','?')})")
                if paths:
                    for i, path in enumerate(paths):
                        print(f"  Path {i+1}: Total Time={path['total_time']:.0f} min, Min Capacity={path['min_capacity']:.2f} kg, Flights={path['flight_ids']}")
                else:
                    print("  No paths found.")
                print("-" * 15)
            elif output_count == 5:
                 print("\n... (output truncated) ...")
                 break
            output_count += 1

        print(f"\nFound path results for {len(k_shortest_paths)} ODTs.")

    except ValueError as e:
        print(f"\nError during processing: {e}")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")
        import traceback
        traceback.print_exc()
else:
    print("\nError: Could not read valid schedule or demand data.")

Starting flight path analysis (Optimized Flight-Vertex Model)...
Capacity data read: 2057 rows.
Market data read: 3185 rows.

Initializing Optimized Flight-Vertex Network...
Precomputing lookups...
Building optimized flight-vertex graph...
1. Adding Source -> Flight edges...
   Added 22420 source edges.
2. Adding Flight -> Flight edges...
   Added 23969 connection edges.
   Added 1670 'waiting' edges.
3. Adding Flight -> Sink edges...
   Added 121331 flight-to-sink edges.
Graph building completed in 0.16 seconds.
Graph nodes estimated around: 8427

Finding 3 shortest paths (Flight-Vertex Model, ranked by user_cost)...
NOTE: Pathfinding uses user-defined costs. Ranking is based on the sum, which equals total duration.
      Graph pruning (first unique OD) may exclude globally optimal paths.
  Processed 100/3185 demands... Elapsed: 1.57s
  Processed 200/3185 demands... Elapsed: 3.64s
  Processed 300/3185 demands... Elapsed: 5.66s
  Processed 400/3185 demands... Elapsed: 7.99s
  Processed

In [46]:
# --- Flight-Vertex Network Class ---
class FlightVertexNetwork:
    def __init__(self, schedule_df, demand_df, k_paths, min_connect_mins):
        print("\nInitializing Flight-Vertex Network...")
        if schedule_df.empty or demand_df.empty:
            raise ValueError("Schedule or Demand dataframe is empty.")

        self.k_paths = k_paths
        self.min_connect_mins = min_connect_mins

        # --- Essential Lookups ---
        print("Precomputing lookups...")
        self.flight_data = schedule_df.set_index('flight_id').to_dict('index')
        self.demand_details = demand_df.set_index('ODT').to_dict('index') # ODT -> {ori, des, time, demand}

        # Group flights by (ori, des) and sort by departure time
        self.flights_by_od = defaultdict(list)
        schedule_sorted_od = schedule_df.sort_values(by=['ori', 'des', 'dep_time'])
        for idx, row in schedule_sorted_od.iterrows():
            self.flights_by_od[(row['ori'], row['des'])].append(row['flight_id'])

        # Group flights by origin, sorted by departure (needed for connections)
        self.flights_by_origin = defaultdict(list)
        schedule_sorted_ori = schedule_df.sort_values(by=['ori', 'dep_time'])
        for idx, row in schedule_sorted_ori.iterrows():
             self.flights_by_origin[row['ori']].append(row['flight_id'])

        # --- Graph Representation (Adjacency List) ---
        # Node -> list of outgoing edges
        # Node types: ('source', odt_str), flight_id_int, ('sink', odt_str)
        # Edge dict: {'neighbor': node, 'cost': float (user_defined!), 'flight_id': int|None, 'is_waiting_edge': bool}
        # We also need to store extra info needed by Dijkstra/Path formatting
        # Edge dict: {'neighbor': node, 'cost': user_cost, 'arr_time': arrival time if neighbor is flight, 'capacity': capacity if neighbor is flight, 'is_waiting_edge': bool}
        self.graph = defaultdict(list)
        self.build_graph()

    def _add_edge(self, u, v, user_cost, arr_time=None, capacity=None, is_waiting_edge=False):
        """Adds edge with user-defined cost and necessary time/capacity info."""
        if u is None or v is None: return

        # Ensure capacity is float/inf
        cap = float(capacity) if isinstance(capacity, (int, float)) and math.isfinite(capacity) else float('inf')

        self.graph[u].append({
            'neighbor': v,
            'cost': float(user_cost), # The cost metric requested by the user
            'arr_time': arr_time,     # Actual arrival time (needed for time checks)
            'capacity': cap,
            'is_waiting_edge': is_waiting_edge # Flag for the ambiguous waiting edge
        })

    def find_first_flight_after(self, flight_list, min_departure_time):
        """Finds the first flight_id in a sorted list departing at or after min_departure_time."""
        # Simple linear scan for now, could use binary search (bisect_left) if lists are very long
        for flight_id in flight_list:
            if self.flight_data[flight_id]['dep_time'] >= min_departure_time:
                return flight_id
        return None

    def build_graph(self):
        """Builds the graph based on user's flight-vertex model rules."""
        print("Building graph based on Flight-Vertex model...")
        start_time = timer.time()

        # --- Node Definitions ---
        # ('source', odt_str)
        # flight_id_int
        # ('sink', odt_str)  <- REQUIRED for per-demand KSP

        # 1. Source to Flight Edges
        print("1. Adding Source -> Flight edges...")
        source_edge_count = 0
        processed_first_ods = {} # ('source', odt_id) -> set((ori, des))
        for odt_id, demand in self.demand_details.items():
            source_node = ('source', odt_id)
            demand_ori = demand['ori']
            demand_time = demand['time']
            processed_first_ods[source_node] = set()

            # Iterate through all OD pairs starting from demand_ori
            for flight_ori, flight_des in [(f['ori'], f['des']) for f in self.flight_data.values() if f['ori'] == demand_ori]:
                 od_pair = (flight_ori, flight_des)
                 if od_pair not in processed_first_ods[source_node]:
                     # Find the *first* flight for this OD pair departing after demand_time
                     relevant_flights = self.flights_by_od.get(od_pair, [])
                     first_flight_id = self.find_first_flight_after(relevant_flights, demand_time)

                     if first_flight_id is not None:
                         flight = self.flight_data[first_flight_id]
                         # User Cost: dep_time - ready_time
                         user_cost = flight['dep_time'] - demand_time
                         # Add edge: Source -> flight_id
                         self._add_edge(source_node, first_flight_id, user_cost,
                                        arr_time=flight['arr_time'], # Store actual arrival time
                                        capacity=flight['cap_kg'])
                         processed_first_ods[source_node].add(od_pair)
                         source_edge_count += 1
        print(f"   Added {source_edge_count} source edges (to first unique OD flights).")


        # 2. Flight to Flight Edges (Connections and Waiting)
        print("2. Adding Flight -> Flight edges...")
        connection_edge_count = 0
        waiting_edge_count = 0
        for flight_id_A, flight_A in self.flight_data.items():
            arrival_airport_A = flight_A['des']
            arrival_time_A = flight_A['arr_time']
            departure_time_A = flight_A['dep_time']
            ori_A, des_A = flight_A['ori'], flight_A['des']

            # a) Connection Edges (A -> B)
            processed_first_conn_ods = set() # Track first unique OD for connections from A
            # Iterate through all flights potentially departing from A's destination
            for flight_id_B in self.flights_by_origin.get(arrival_airport_A, []):
                 flight_B = self.flight_data[flight_id_B]
                 od_pair_B = (flight_B['ori'], flight_B['des'])

                 # Check temporal constraint first
                 if flight_B['dep_time'] >= arrival_time_A + self.min_connect_mins:
                     # Check if we already added an edge for this unique OD pair from flight_A
                     if od_pair_B not in processed_first_conn_ods:
                          # User Cost: B.dep_time - A.dep_time
                          user_cost = flight_B['dep_time'] - departure_time_A
                          self._add_edge(flight_id_A, flight_id_B, user_cost,
                                         arr_time=flight_B['arr_time'], # Store B's arrival time
                                         capacity=flight_B['cap_kg'])
                          processed_first_conn_ods.add(od_pair_B)
                          connection_edge_count += 1
                 # Optimization: Since flights_by_origin is sorted, if B is too early,
                 # subsequent flights might be okay. But if B is valid, we only take the first per OD.

            # b) Waiting Edges (A -> C, same OD)
            flights_same_od = self.flights_by_od.get((ori_A, des_A), [])
            # Find the index of flight_A in this list
            try:
                idx_A = flights_same_od.index(flight_id_A)
                # Check if there is a *next* flight in the list
                if idx_A + 1 < len(flights_same_od):
                    flight_id_C = flights_same_od[idx_A + 1]
                    flight_C = self.flight_data[flight_id_C]
                    # User Cost: C.dep_time - A.dep_time
                    user_cost = flight_C['dep_time'] - departure_time_A
                    # Add edge A -> C, marking it as a waiting edge
                    self._add_edge(flight_id_A, flight_id_C, user_cost,
                                   arr_time=flight_C['arr_time'], # Store C's arrival time
                                   capacity=flight_C['cap_kg'], # Capacity of C is relevant?
                                   is_waiting_edge=True)
                    waiting_edge_count += 1
            except ValueError:
                # flight_id_A not found in its own OD list? Should not happen.
                pass
        print(f"   Added {connection_edge_count} connection edges (to first unique OD).")
        print(f"   Added {waiting_edge_count} 'waiting' edges (to next same OD flight).")

        # 3. Flight to Sink Edges
        print("3. Adding Flight -> Sink edges...")
        sink_edge_count = 0
        for flight_id, flight in self.flight_data.items():
            dest_airport = flight['des']
            # Connect to ALL demand sinks that require this destination
            # This requires iterating through demands again, maybe inefficient
            # Alternative: Create sink nodes first?
            for odt_id, demand in self.demand_details.items():
                if demand['des'] == dest_airport:
                     sink_node = ('sink', odt_id)
                     # User Cost: Duration of flight F
                     user_cost = flight['arr_time'] - flight['dep_time']
                     # Need flight's arrival time to calculate final duration
                     self._add_edge(flight_id, sink_node, user_cost,
                                    arr_time=flight['arr_time']) # Store arrival time
                     sink_edge_count += 1
        print(f"   Added {sink_edge_count} flight-to-sink edges.")


        print(f"Graph building completed in {timer.time() - start_time:.2f} seconds.")
        est_nodes = len(self.demand_details) + len(self.flight_data) + len(self.demand_details) # sources + flights + sinks
        print(f"Graph nodes estimated around: {est_nodes}")


    def _dijkstra(self, start_node, end_node, forbidden_edges=None, forbidden_nodes=None):
        """
        Modified Dijkstra for the Flight-Vertex model.
        Tracks user_cost for minimization and actual_time for validation.
        Returns (min_user_cost, final_actual_arrival_time, path_list)
        """
        global tie_breaker
        if forbidden_edges is None: forbidden_edges = set()
        if forbidden_nodes is None: forbidden_nodes = set()

        # Get demand ready time from start_node
        demand_ready_time = self.demand_details[start_node[1]]['time']

        # Priority Queue: (user_cost, tie_id, current_actual_arr_time, current_node, path_list)
        # Path List: [{'node': node, 'edge_info': edge_dict_or_None}]
        # current_actual_arr_time is the arrival time at current_node (if it's a flight) or ready_time (if source)
        pq = [(0.0, next(tie_breaker), demand_ready_time, start_node, [{'node': start_node, 'edge_info': None}])]

        # Visited stores the *minimum user_cost* found to reach a node
        visited_user_costs = defaultdict(lambda: float('inf'))
        visited_user_costs[start_node] = 0.0
        # We also need to track min arrival time to handle cycles correctly with time check
        # Note: This makes the state space complex again, similar to time-expansion!
        min_arrival_time_at_node = defaultdict(lambda: float('inf'))
        min_arrival_time_at_node[start_node] = demand_ready_time


        final_arrival_time = float('inf') # Track the actual arrival time for the best path found so far

        while pq:
            user_cost, _, current_actual_time, current_node, path_list = heapq.heappop(pq)

            # Pruning based on user_cost
            if user_cost > visited_user_costs[current_node]:
                continue

            # Check if reached sink
            if current_node == end_node:
                # We found *a* path. Dijkstra guarantees this is the path with
                # the minimum *user_cost*. Store its actual arrival time.
                # The *true* shortest path in terms of duration might be found later
                # if its user_cost is higher but arrival time is earlier.
                # This basic Dijkstra finds only the min-user-cost path.
                # Need modification for KSP based on actual time. Let's return the first found path for now.
                final_arrival_time = current_actual_time # Time of arrival at the node *before* the sink
                return user_cost, final_arrival_time, path_list


            # Explore neighbors
            if current_node not in self.graph: continue

            for edge in self.graph[current_node]:
                neighbor_node = edge['neighbor']
                edge_user_cost = edge['cost']
                neighbor_actual_arr_time = edge['arr_time'] # Actual arrival time provided by edge
                is_waiting_edge = edge['is_waiting_edge']

                # --- Forbidden checks ---
                if neighbor_node in forbidden_nodes: continue
                # Edge representation for forbidden check: (from_node, to_node, neighbor_flight_id_or_None)
                # Neighbor flight ID is the node itself if it's a flight_id node
                neighbor_flight_id = neighbor_node if isinstance(neighbor_node, int) else None
                edge_rep = (current_node, neighbor_node, neighbor_flight_id)
                if edge_rep in forbidden_edges: continue
                # --- End Forbidden ---

                # --- **CRUCIAL TIME VALIDATION** ---
                # This check MUST use actual time, simulating time-expansion logic
                valid_connection = False
                if isinstance(current_node, int): # If current node is a flight
                    flight_A_data = self.flight_data[current_node]
                    # Check if neighbor is a flight and connection time is valid
                    if isinstance(neighbor_node, int):
                        flight_B_data = self.flight_data[neighbor_node]
                        # Use the actual arrival time at current_node for check
                        if flight_B_data['dep_time'] >= current_actual_time + self.min_connect_mins:
                            valid_connection = True
                    elif isinstance(neighbor_node, tuple) and neighbor_node[0] == 'sink':
                        valid_connection = True # Flight -> Sink is always valid temporally
                elif isinstance(current_node, tuple) and current_node[0] == 'source':
                    # Source -> Flight is always valid temporally by graph construction rule
                     valid_connection = True
                # Add logic for waiting edge? If A->C wait edge, when is C valid?
                # Assuming C is valid if C.dep_time > A.dep_time (checked in build_graph)
                if is_waiting_edge:
                     valid_connection = True # Temporal validity checked during build


                if not valid_connection:
                    continue
                # --- End Time Validation ---


                new_user_cost = user_cost + edge_user_cost

                # Relaxation based on user_cost
                # Also consider actual arrival time to prevent suboptimal cycles in time
                if new_user_cost < visited_user_costs[neighbor_node] or \
                   (neighbor_actual_arr_time is not None and neighbor_actual_arr_time < min_arrival_time_at_node[neighbor_node]):

                    visited_user_costs[neighbor_node] = new_user_cost
                    if neighbor_actual_arr_time is not None:
                         min_arrival_time_at_node[neighbor_node] = neighbor_actual_arr_time

                    new_step = {'node': neighbor_node, 'edge_info': edge}
                    new_path = path_list + [new_step]
                    heapq.heappush(pq, (new_user_cost, next(tie_breaker), neighbor_actual_arr_time, neighbor_node, new_path))

        # If end_node wasn't reached
        return float('inf'), float('inf'), None


    def _format_path(self, path_list, final_arrival_time, demand_ready_time):
        """Formats path, calculating true duration."""
        if not path_list or len(path_list) < 2: return None

        flight_ids = []
        min_capacity = float('inf')
        actual_nodes_taken = [] # Track nodes excluding source/sink maybe

        for step in path_list[1:]: # Skip source
            node = step['node']
            edge = step['edge_info']
            actual_nodes_taken.append(node)

            if isinstance(node, int) and edge and not edge['is_waiting_edge']: # If it's a flight node reached via non-waiting edge
                 flight_ids.append(node)
                 min_capacity = min(min_capacity, edge['capacity'])
            # How to handle capacity if a waiting edge A->C was taken? Does A's capacity count? Or C's?
            # Assuming C's capacity is stored on the waiting edge info.
            elif isinstance(node, int) and edge and edge['is_waiting_edge']:
                 flight_ids.append(node) # Add the flight C we waited *for*
                 min_capacity = min(min_capacity, edge['capacity']) # Use C's capacity

        min_cap_final = 0.0 if min_capacity == float('inf') else min_capacity

        # **Calculate True Duration**
        if final_arrival_time == float('inf'):
             true_duration = float('inf')
        else:
            # final_arrival_time is the arrival time of the *last flight* before the sink
             true_duration = final_arrival_time - demand_ready_time

        return {
            'total_time': true_duration, # Actual elapsed duration
            'flight_ids': flight_ids,    # Flights considered "taken"
            'min_capacity': min_cap_final,
            # 'internal_cost': user_cost # Can include for debugging
            # 'nodes': actual_nodes_taken # Can include for debugging
        }


    def find_k_shortest_paths(self):
        """Finds K shortest paths using Yen's, ranking by TRUE DURATION."""
        print(f"\nFinding {self.k_paths} shortest paths (ranking by true duration)...")
        print("WARNING: Underlying graph uses user-defined costs for pathfinding,")
        print("         which may not align with minimizing true duration.")

        all_results = {}
        total_demands = len(self.demand_details)
        start_run_time = timer.time()

        for i, (odt_id, demand) in enumerate(self.demand_details.items()):
            if (i + 1) % 100 == 0:
                elapsed = timer.time() - start_run_time
                print(f"  Processed {i+1}/{total_demands} demands... Elapsed: {elapsed:.2f}s")

            start_node = ('source', odt_id)
            end_node = ('sink', odt_id) # MUST be demand-specific sink
            demand_ready_time = demand['time']

            A = [] # Stores final paths: (true_duration, user_cost, path_list) - Sort by true_duration!
            B_candidates = [] # Candidate heap: (user_cost, tie_id, actual_arrival_time, path_list) - Heap by user_cost

            # 1. Find first shortest path (based on user_cost)
            user_cost1, arrival1, path1 = self._dijkstra(start_node, end_node)

            if path1:
                duration1 = arrival1 - demand_ready_time if arrival1 != float('inf') else float('inf')
                if duration1 != float('inf'):
                     A.append((duration1, user_cost1, path1))
                # Cannot add to B yet
            else:
                all_results[odt_id] = []
                continue

            # Cache for paths found (use node sequence tuple as key) to avoid adding duplicates to A
            # Store the user_cost associated with the path found for that sequence
            found_paths_cache = {}
            if path1:
                 found_paths_cache[tuple(s['node'] for s in path1)] = user_cost1


            # 2. Find paths k = 2 to K
            for k in range(1, self.k_paths):
                if k > len(A): break

                # Get the (k-1)th path *based on true duration ranking*
                # A is sorted by duration, so A[k-1] is correct conceptually
                prev_duration, prev_user_cost, prev_path = A[k-1]

                # Iterate through nodes of path k-1 to find spur points
                current_path_actual_time = demand_ready_time
                for i in range(len(prev_path) - 1): # Spur node cannot be sink
                    spur_node = prev_path[i]['node']
                    spur_node_step = prev_path[i]
                    edge_to_spur = spur_node_step['edge_info']

                    # Update actual time at spur node
                    if edge_to_spur:
                         # Need the 'arr_time' from the edge LEADING TO the spur node
                         # This requires looking at the edge info in the *current* step i
                         current_path_actual_time = edge_to_spur.get('arr_time', current_path_actual_time) # Update time


                    root_path = prev_path[:i+1]
                    # Calculate root user_cost
                    root_user_cost = sum(s['edge_info']['cost'] for s in root_path[1:] if s['edge_info'])

                    forbidden_nodes = {step['node'] for step in root_path[:-1]}
                    forbidden_edges = set()

                    # Forbid edges leaving spur node used by paths in A sharing the same root
                    for dur_j, cost_j, path_j in A:
                        if len(path_j) > i + 1:
                            if [step['node'] for step in path_j[:i+1]] == [step['node'] for step in root_path]:
                                edge_info_j = path_j[i+1]['edge_info']
                                if edge_info_j:
                                     neighbor_j = edge_info_j['neighbor']
                                     neighbor_flight_id_j = neighbor_j if isinstance(neighbor_j, int) else None
                                     edge_rep = (spur_node, neighbor_j, neighbor_flight_id_j)
                                     forbidden_edges.add(edge_rep)

                    # --- Calculate spur path using _dijkstra ---
                    # Need to run Dijkstra *from* the spur node with its *correct actual time*
                    # Our current _dijkstra starts from ready_time. This requires modification
                    # or a separate function that takes start_time.
                    # Q: Can we reuse _dijkstra if we recalculate costs relative to spur? Maybe not easily.

                    # --- SIMPLIFICATION FOR NOW: Rerun Dijkstra from global start with forbids ---
                    # This is inefficient but follows Yen's structure more easily here.
                    spur_user_cost, spur_arrival, spur_path_list = self._dijkstra(start_node, end_node, forbidden_edges, forbidden_nodes)

                    # If a new path is found *from the original source*
                    if spur_path_list:
                        # Check if this path is genuinely new (different node sequence)
                        spur_nodes_tuple = tuple(s['node'] for s in spur_path_list)
                        # If this path wasn't found before OR if found with higher user_cost
                        if spur_nodes_tuple not in found_paths_cache or spur_user_cost < found_paths_cache[spur_nodes_tuple]:
                            spur_duration = spur_arrival - demand_ready_time if spur_arrival != float('inf') else float('inf')
                            if spur_duration != float('inf'):
                                 # Add to candidate list B (heap not strictly needed if we re-sort later)
                                 # Store (duration, user_cost, path_list) for candidate
                                 B_candidates.append((spur_duration, spur_user_cost, spur_path_list))
                                 found_paths_cache[spur_nodes_tuple] = spur_user_cost # Update cache


                # Select the best path from B_candidates based on true duration that isn't already in A
                if not B_candidates:
                     break # No more potential paths found

                # Sort candidates by true duration
                B_candidates.sort(key=lambda x: x[0])

                found_next = False
                processed_candidates = []
                for cand_duration, cand_user_cost, cand_path in B_candidates:
                    cand_nodes = tuple(s['node'] for s in cand_path)
                    is_in_A = any(tuple(s['node'] for s in path_a) == cand_nodes for dur_a, cost_a, path_a in A)
                    if not is_in_A:
                        A.append((cand_duration, cand_user_cost, cand_path))
                        # Remove this candidate from B list for next iteration
                        # B_candidates.remove((cand_duration, cand_user_cost, cand_path)) # This is slow
                        found_next = True
                        break # Found path k
                    else:
                        # Keep track to rebuild B for the next k without duplicates already added to A
                         processed_candidates.append((cand_duration, cand_user_cost, cand_path))

                # Rebuild B for next iteration, removing paths just added to A
                new_B = []
                existing_A_nodes = {tuple(s['node'] for s in p) for d, c, p in A}
                for cand_d, cand_c, cand_p in B_candidates:
                     if tuple(s['node'] for s in cand_p) not in existing_A_nodes:
                          new_B.append((cand_d, cand_c, cand_p))
                B_candidates = new_B


                if not found_next: break # No more new paths found

            # Format results for ODT, A is already sorted by duration
            formatted_paths = []
            for duration, user_cost, path_list in A:
                 formatted = self._format_path(path_list, duration + demand_ready_time, demand_ready_time) # Pass final arrival time
                 if formatted:
                     formatted['internal_cost'] = user_cost # Add user cost for info
                     formatted_paths.append(formatted)
            all_results[odt_id] = formatted_paths[:self.k_paths] # Ensure only K paths are stored


        print(f"\nFinished finding K shortest paths in {timer.time() - start_run_time:.2f} seconds.")
        return all_results


# --- Main Execution Logic ---
print("Starting flight path analysis (Flight-Vertex Model)...")

# 1. Read Data
schedule_data = read_capacity(CAPACITY_FILE, airport_substitutions)
demand_data = read_market(MARKET_FILE, airport_substitutions)

# 2. Initialize and Run only if data is valid
if not schedule_data.empty and not demand_data.empty:
    try:
        flight_network = FlightVertexNetwork(schedule_data, demand_data, K_PATHS, MIN_CONNECT_MINS)
        k_shortest_paths = flight_network.find_k_shortest_paths()

        # 3. Output Sample Results
        print("\n--- K Shortest Paths Results (Sample - Ranked by True Duration) ---")
        output_count = 0
        for odt, paths in k_shortest_paths.items():
            if output_count < 5:
                demand_info = flight_network.demand_details.get(odt, {})
                print(f"ODT: {odt} (Demand: {demand_info.get('ori','?')}-{demand_info.get('des','?')} @ {demand_info.get('time','?')})")
                if paths:
                    for i, path in enumerate(paths):
                        print(f"  Path {i+1}: True Duration={path['total_time']:.0f} min, Min Capacity={path['min_capacity']:.2f} kg, Flights={path['flight_ids']} (UserCost={path.get('internal_cost',-1):.0f})")
                else:
                    print("  No paths found.")
                print("-" * 15)
            elif output_count == 5:
                 print("\n... (output truncated) ...")
                 break
            output_count += 1

        print(f"\nFound path results for {len(k_shortest_paths)} ODTs.")

    except ValueError as e:
        print(f"\nError during processing: {e}")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")
        import traceback
        traceback.print_exc()
else:
    print("\nError: Could not read valid schedule or demand data.")

Starting flight path analysis (Flight-Vertex Model)...
Capacity data read: 2057 rows.
Market data read: 3185 rows.

Initializing Flight-Vertex Network...
Precomputing lookups...
Building graph based on Flight-Vertex model...
1. Adding Source -> Flight edges...
   Added 22420 source edges (to first unique OD flights).
2. Adding Flight -> Flight edges...
   Added 23969 connection edges (to first unique OD).
   Added 1670 'waiting' edges (to next same OD flight).
3. Adding Flight -> Sink edges...
   Added 121331 flight-to-sink edges.
Graph building completed in 0.41 seconds.
Graph nodes estimated around: 8427

Finding 3 shortest paths (ranking by true duration)...
         which may not align with minimizing true duration.
  Processed 100/3185 demands... Elapsed: 2.12s
  Processed 200/3185 demands... Elapsed: 5.07s
  Processed 300/3185 demands... Elapsed: 7.89s
  Processed 400/3185 demands... Elapsed: 11.22s
  Processed 500/3185 demands... Elapsed: 15.44s
  Processed 600/3185 demands... E

In [31]:
# --- Flight Network Class (Refined) ---
class FlightNetwork:
    def __init__(self, schedule_df, demand_df, k_paths, min_connect_mins):
        print("\nInitializing Flight Network...")
        if schedule_df.empty or demand_df.empty:
             raise ValueError("Schedule or Demand dataframe is empty.")

        self.k_paths = k_paths
        self.min_connect_mins = min_connect_mins

        # Efficient Lookups (using integer flight_id)
        print("Precomputing lookups...")
        self.flight_data = schedule_df.set_index('flight_id').to_dict('index')
        self.flights_by_origin = schedule_df.sort_values(by='dep_time').groupby('ori')['flight_id'].apply(list).to_dict()
        self.demand_dict = demand_df.set_index('ODT').to_dict('index')

        # Graph Representation: Adjacency list
        # node -> list_of_outgoing_edges
        # edge = {'neighbor': node_tuple, 'weight': float, 'flight_id': int|None, 'capacity': float, 'is_flight': bool}
        self.graph = defaultdict(list)
        self.build_graph()

    def _add_edge(self, u, v, weight, flight_id=None, capacity=float('inf'), is_flight=False):
        """Adds a directed edge if weight is valid."""
        if weight < 0: return # Skip invalid edges

        # Ensure capacity is float/inf
        cap = float(capacity) if isinstance(capacity, (int, float)) and math.isfinite(capacity) else float('inf')

        self.graph[u].append({
            'neighbor': v,
            'weight': float(weight), # Use float for time
            'flight_id': flight_id, # Int or None
            'capacity': cap,
            'is_flight': is_flight
        })

    def build_graph(self):
        """Builds the time-expanded graph."""
        print("Building graph...")
        start_time = timer.time()

        # --- Node Definitions ---
        # ('source', odt_str) : Start node for a specific demand ODT
        # ('sink', odt_str)   : End node for a specific demand ODT
        # (flight_id_int, 'dep') : Departure event of a flight
        # (flight_id_int, 'arr') : Arrival event of a flight

        # 1. Flight Edges (Dep -> Arr)
        for flight_id, flight in self.flight_data.items():
            u_node = (flight_id, 'dep')
            v_node = (flight_id, 'arr')
            duration = flight['arr_time'] - flight['dep_time']
            self._add_edge(u_node, v_node, duration, flight_id, flight['cap_kg'], is_flight=True)

        # Pre-group demands by destination for sink edge efficiency
        demands_by_dest = defaultdict(list)
        for odt_id, demand_info in self.demand_dict.items():
             demands_by_dest[demand_info['des']].append(odt_id)

        # 2. Connection (Arr -> Dep) and Sink (Arr -> Sink) Edges
        for flight1_id, flight1 in self.flight_data.items():
            arrival_node = (flight1_id, 'arr')
            arrival_time = flight1['arr_time']
            arrival_airport = flight1['des']

            # a) Sink Edges: Connect to sinks of demands ending here
            for odt_id in demands_by_dest.get(arrival_airport, []):
                 sink_node = ('sink', odt_id)
                 self._add_edge(arrival_node, sink_node, weight=0, is_flight=False)

            # b) Connection Edges: Connect to valid next flights
            for flight2_id in self.flights_by_origin.get(arrival_airport, []):
                flight2 = self.flight_data[flight2_id]
                if flight2['dep_time'] >= arrival_time + self.min_connect_mins:
                    departure_node = (flight2_id, 'dep')
                    wait_time = flight2['dep_time'] - arrival_time
                    self._add_edge(arrival_node, departure_node, weight=wait_time, is_flight=False)
                # Since flights_by_origin lists are sorted, we could potentially break
                # if wait_time gets excessively large, but finding K shortest needs exploration.

        # 3. Source Edges (Source -> Dep)
        for odt_id, demand in self.demand_dict.items():
            source_node = ('source', odt_id)
            origin_airport = demand['ori']
            ready_time = demand['time']

            for flight_id in self.flights_by_origin.get(origin_airport, []):
                flight = self.flight_data[flight_id]
                if flight['dep_time'] >= ready_time:
                    departure_node = (flight_id, 'dep')
                    wait_time = flight['dep_time'] - ready_time
                    self._add_edge(source_node, departure_node, weight=wait_time, is_flight=False)
                # else: flight departs too early

        print(f"Graph building completed in {timer.time() - start_time:.2f} seconds.")
        est_nodes = len(self.demand_dict) * 2 + len(self.flight_data) * 2
        print(f"Graph nodes estimated around: {est_nodes}")


    def _dijkstra(self, start_node, end_node, forbidden_edges=None, forbidden_nodes=None):
        """Finds shortest path using Dijkstra with tie-breaker."""
        global tie_breaker
        if forbidden_edges is None: forbidden_edges = set()
        if forbidden_nodes is None: forbidden_nodes = set()

        # Priority Queue: (cost, tie_id, current_node, path_list)
        # path_list = [{'node': node_tuple, 'edge_info': edge_dict_or_None}, ...]
        pq = [(0.0, next(tie_breaker), start_node, [{'node': start_node, 'edge_info': None}])]
        visited_costs = defaultdict(lambda: float('inf'))
        visited_costs[start_node] = 0.0

        while pq:
            cost, _, current_node, path_list = heapq.heappop(pq)

            if cost > visited_costs[current_node]: continue
            if current_node == end_node: return cost, path_list

            if current_node not in self.graph: continue # Sink nodes have no outgoing

            for edge in self.graph[current_node]:
                neighbor = edge['neighbor']
                if neighbor in forbidden_nodes: continue

                # Edge representation for forbidden check: (from_node, to_node, flight_id_or_None)
                edge_rep = (current_node, neighbor, edge['flight_id'])
                if edge_rep in forbidden_edges: continue

                new_cost = cost + edge['weight']
                if new_cost < visited_costs[neighbor]:
                    visited_costs[neighbor] = new_cost
                    new_step = {'node': neighbor, 'edge_info': edge}
                    new_path = path_list + [new_step]
                    heapq.heappush(pq, (new_cost, next(tie_breaker), neighbor, new_path))

        return float('inf'), None


    def _format_path(self, path_list, total_cost):
        """Formats Dijkstra/Yen's path list into desired output."""
        if not path_list or len(path_list) < 2: return None

        flight_ids = []
        min_capacity = float('inf')

        for step in path_list[1:]: # Skip source node
            edge = step['edge_info']
            if edge and edge['is_flight']:
                flight_ids.append(edge['flight_id'])
                min_capacity = min(min_capacity, edge['capacity'])

        # If min_capacity is still inf (no flights), set to 0.0
        min_cap_final = 0.0 if min_capacity == float('inf') else min_capacity

        # total_cost from Dijkstra *is* (arrival_at_sink - demand_ready_time)
        return {
            'total_time': total_cost,
            'flight_ids': flight_ids,
            'min_capacity': min_cap_final
        }


    def find_k_shortest_paths(self):
        """Finds K shortest paths for each demand using Yen's."""
        print(f"\nFinding {self.k_paths} shortest paths for {len(self.demand_dict)} demands...")
        all_results = {}
        total_demands = len(self.demand_dict)
        start_run_time = timer.time()

        for i, (odt_id, demand) in enumerate(self.demand_dict.items()):
            if (i + 1) % 100 == 0:
                elapsed = timer.time() - start_run_time
                print(f"  Processed {i+1}/{total_demands} demands... Elapsed: {elapsed:.2f}s")

            start_node = ('source', odt_id)
            end_node = ('sink', odt_id)

            A = [] # Stores final paths: (cost, path_list)
            B = [] # Candidate heap: (cost, tie_id, path_list)

            # 1. Find first shortest path
            cost1, path1 = self._dijkstra(start_node, end_node)
            if path1:
                A.append((cost1, path1))
            else:
                all_results[odt_id] = []
                continue # No path exists

            # 2. Find paths k = 2 to K
            for k in range(1, self.k_paths):
                if k > len(A): break # Cannot find k if k-1 doesn't exist

                prev_cost, prev_path = A[k-1] # Path k-1 (0-indexed)

                # Iterate through nodes of path k-1 to find spur points
                for i in range(len(prev_path) - 1): # Spur node cannot be sink
                    spur_node = prev_path[i]['node']
                    root_path = prev_path[:i+1]
                    root_cost = sum(s['edge_info']['weight'] for s in root_path[1:] if s['edge_info'])

                    forbidden_nodes = {step['node'] for step in root_path[:-1]}
                    forbidden_edges = set()

                    # Forbid edges leaving spur node used by previous paths (A[0]..A[k-1]) sharing the same root
                    for cost_j, path_j in A: # Check all paths found so far in A
                        if len(path_j) > i + 1:
                            # Check if node sequence of root matches
                            if [step['node'] for step in path_j[:i+1]] == [step['node'] for step in root_path]:
                                edge_info_j = path_j[i+1]['edge_info']
                                if edge_info_j:
                                    edge_rep = (spur_node, edge_info_j['neighbor'], edge_info_j['flight_id'])
                                    forbidden_edges.add(edge_rep)

                    # Calculate spur path
                    spur_cost_delta, spur_path_segment = self._dijkstra(spur_node, end_node, forbidden_edges, forbidden_nodes)

                    if spur_path_segment:
                        # Combine root and spur
                        full_path = root_path + spur_path_segment[1:]
                        total_new_cost = root_cost + spur_cost_delta
                        # Use tuple representation of nodes for quick duplicate check in B (optional but good practice)
                        # path_node_tuple = tuple(item['node'] for item in full_path)
                        heapq.heappush(B, (total_new_cost, next(tie_breaker), full_path))

                # Find best candidate in B not already in A
                found_next = False
                while B:
                    potential_cost, _, potential_path = heapq.heappop(B)
                    # Check if path (by node sequence) is already in A
                    potential_nodes = tuple(item['node'] for item in potential_path)
                    is_in_A = any(tuple(item['node'] for item in path_a) == potential_nodes for cost_a, path_a in A)

                    if not is_in_A:
                        A.append((potential_cost, potential_path))
                        found_next = True
                        break # Found path k

                if not found_next: break # No more paths found for this demand

            # Format results for ODT
            formatted_paths = []
            A.sort(key=lambda x: x[0]) # Ensure sorted output
            for cost, path_list in A:
                 formatted = self._format_path(path_list, cost)
                 if formatted: formatted_paths.append(formatted)
            all_results[odt_id] = formatted_paths

        print(f"\nFinished finding K shortest paths in {timer.time() - start_run_time:.2f} seconds.")
        return all_results


# --- Main Execution Logic ---
print("Starting flight path analysis...")

# 1. Read Data
schedule_data = read_capacity(CAPACITY_FILE, airport_substitutions)
demand_data = read_market(MARKET_FILE, airport_substitutions)
#print(demand_data.head())

# 2. Initialize and Run only if data is valid
if not schedule_data.empty and not demand_data.empty:
    try:
        flight_network = FlightNetwork(schedule_data, demand_data, K_PATHS, MIN_CONNECT_MINS)
        k_shortest_paths = flight_network.find_k_shortest_paths()

        # 3. Output Sample Results
        print("\n--- K Shortest Paths Results (Sample) ---")
        output_count = 0
        for odt, paths in k_shortest_paths.items():
            if output_count < 5: # Print details for first 5 demands
                demand_info = flight_network.demand_dict.get(odt, {})
                print(f"ODT: {odt} (Demand: {demand_info.get('ori','?')}-{demand_info.get('des','?')} @ {demand_info.get('time','?')})")
                if paths:
                    for i, path in enumerate(paths):
                        # The 'total_time' is already (arrival - ready_time)
                        print(f"  Path {i+1}: Total Duration={path['total_time']:.0f} min, Min Capacity={path['min_capacity']:.2f} kg, Flights={path['flight_ids']}")
                else:
                    print("  No paths found.")
                print("-" * 15)
            elif output_count == 5:
                 print("\n... (output truncated) ...")
                 break
            output_count += 1

        print(f"\nFound path results for {len(k_shortest_paths)} ODTs.")
        # 'k_shortest_paths' dictionary holds all results

    except ValueError as e:
        print(f"\nError during processing: {e}")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")
        import traceback
        traceback.print_exc()
else:
    print("\nError: Could not read valid schedule or demand data.")

Starting flight path analysis...
Capacity data read: 2057 rows.
Market data read: 3185 rows.

Initializing Flight Network...
Precomputing lookups...
Building graph...
Graph building completed in 0.32 seconds.
Graph nodes estimated around: 10484

Finding 3 shortest paths for 3185 demands...
  Processed 100/3185 demands... Elapsed: 0.94s
  Processed 200/3185 demands... Elapsed: 2.18s
  Processed 300/3185 demands... Elapsed: 3.37s
  Processed 400/3185 demands... Elapsed: 4.72s
  Processed 500/3185 demands... Elapsed: 6.64s
  Processed 600/3185 demands... Elapsed: 7.67s
  Processed 700/3185 demands... Elapsed: 8.78s
  Processed 800/3185 demands... Elapsed: 10.34s
  Processed 900/3185 demands... Elapsed: 10.97s
  Processed 1000/3185 demands... Elapsed: 11.85s
  Processed 1100/3185 demands... Elapsed: 12.89s
  Processed 1200/3185 demands... Elapsed: 14.46s
  Processed 1300/3185 demands... Elapsed: 16.30s
  Processed 1400/3185 demands... Elapsed: 17.82s
  Processed 1500/3185 demands... Elapse

In [45]:
paths_df = pd.DataFrame.from_dict(k_shortest_paths, orient='index')
paths_df[0][0], flight_network.flight_data[536]


/var/folders/84/sbq4d0yx2vdcrx0_5_cy2hmh0000gn/T/ipykernel_21888/3429030786.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  paths_df[0][0], flight_network.flight_data[536]


({'total_time': 1850.0,
  'flight_ids': [24, 1306, 1685, 536],
  'min_capacity': 47650.0},
 {'flight_number': 'GB880',
  'ori': 'CVG',
  'des': 'LAX',
  'aircraft_type': '76Y',
  'dep_time': 2640,
  'arr_time': 2930,
  'cap_kg': 47650})

In [25]:
class FlightNetwork:
    def __init__(self, schedule_df, demand_df, K_PATHS, MIN_CONNECT_MINS, MAX_CONNECT_MINS):
        # Settings
        self.k_paths = K_PATHS
        self.min_connect_mins = MIN_CONNECT_MINS
        self.max_connect_mins = MAX_CONNECT_MINS
        
        # DATA
        self.schedule_df = schedule_df
        self.demand_df = demand_df
        self.airports = set(schedule_df['ori']).union(set(schedule_df['des']))

        self.flight_data_dict = dict(zip(schedule_df['flight_id'], schedule_df.to_dict(orient='records')))
        self.flight_capacity_dict = dict(zip(schedule_df['flight_id'], schedule_df['cap_kg']))  
        self.flight_id_to_orides = {row['flight_id']: (row['ori'], row['des']) for _, row in schedule_df.iterrows()}
        self.flight_id_to_dep_arr = {row['flight_id']: (row['dep_time'], row['arr_time']) for _, row in schedule_df.iterrows()}
        self.orides_to_flight_ids = schedule_df.groupby(['ori', 'des'])['flight_id'].apply(list).to_dict()
        self.origin_to_orides = schedule_df.groupby('ori')['des'].apply(list).to_dict()
        
        self.demand_dict = dict(zip(demand_df['ODT'], demand_df.to_dict(orient='records')))
        self.ODT_to_amount = dict(zip(demand_df['ODT'], demand_df['demand']))
        self.ODT_to_orides = {row['ODT']: (row['ori'], row['des']) for _, row in demand_df.iterrows()}
        
    # OFC DONT USE ADJENCY MATRIX BUT FASTER DATASTRUCTURES WITH SIMILAR FUNCTIONALITY
        # Sources to Flights connections - |D| x |F| - demand_id x flight_id - 1 to next, first of unique orides, flights - that start 
        #           from a given origin, else 0
        
        # Flights to Flights connections - |F| x |F| - flight_id x flight_id - 1 to next, first of unique orides, flights 
        #           zero to next same orides flights - free connection - trick to compact the network
        #           -1 if no connection
        
        # Flights to Sinks connections - |F| x |D| - flight_id x demand_id -
        
        # Time Distance Between each airport - |A| x |A| - for aproximation of the time between each airport
        # orides pair to time 
        
        self.build_graph()


    

In [30]:
# --- Flight Network Class (Simplified) ---
class FlightNetwork:
    def __init__(self, schedule_df, demand_df, k_paths, min_connect_mins):
        print("\nInitializing Flight Network...")
        if schedule_df.empty or demand_df.empty:
             raise ValueError("Schedule or Demand dataframe is empty.")

        self.k_paths = k_paths
        self.min_connect_mins = min_connect_mins
        self.schedule_df = schedule_df # Keep original reference if needed
        self.demand_df = demand_df     # Keep original reference if needed

        # --- Precompute Lookups for Efficiency ---
        print("Precomputing flight data lookups...")
        # flight_id (int index) -> flight details dictionary
        self.flight_data = self.schedule_df.set_index('flight_id').to_dict('index')

        # ori -> list of flight_ids sorted by dep_time
        self.flights_by_origin = defaultdict(list)
        # Sort once and group
        schedule_sorted = self.schedule_df.sort_values(by=['ori', 'dep_time'])
        for idx, row in schedule_sorted.iterrows():
            self.flights_by_origin[row['ori']].append(row['flight_id']) # Store flight_id (int)

        # ODT (str) -> demand details dictionary
        self.demand_dict = self.demand_df.set_index('ODT').to_dict('index')

        # --- Build Graph ---
        self.graph = defaultdict(list) # Adjacency list: node -> list of edge dicts
        self.build_graph()

    def _add_edge(self, u, v, weight, flight_id=None, capacity=float('inf'), is_flight=False):
        """Adds a directed edge with attributes to the graph."""
        # Basic check for valid edge
        if weight < 0 or u is None or v is None:
            # print(f"Warning: Invalid edge skipped: {u} -> {v} (Weight: {weight})")
            return

        # Ensure capacity is float or inf
        if not isinstance(capacity, (int, float)):
            capacity = float('inf')
        elif not math.isfinite(capacity):
             capacity = float('inf')

        self.graph[u].append({
            'neighbor': v,          # Target node tuple
            'weight': float(weight),# Time cost (float for precision)
            'flight_id': flight_id, # Integer flight ID or None
            'capacity': capacity,   # Float capacity (or inf)
            'is_flight': is_flight  # Boolean flag
        })

    def build_graph(self):
        """Builds the graph with explicit source/sink per demand."""
        print("Building graph edges...")
        start_time = timer.time()

        # Node Types:
        # ('source', odt_id_str)
        # ('sink', odt_id_str)
        # (flight_id_int, 'dep')
        # (flight_id_int, 'arr')

        # 1. Add Flight Edges (Dep -> Arr)
        print("Adding flight leg edges...")
        flight_edge_count = 0
        for flight_id, flight in self.flight_data.items(): # flight_id is int
            u_node = (flight_id, 'dep')
            v_node = (flight_id, 'arr')
            duration = flight['arr_time'] - flight['dep_time']
            if duration >= 0: # Should be true based on read_capacity filter
                self._add_edge(u_node, v_node, duration, flight_id, flight['cap_kg'], is_flight=True)
                flight_edge_count += 1
        print(f"Added {flight_edge_count} flight leg edges.")

        # Pre-group demands by destination for faster sink edge creation
        demands_by_dest = defaultdict(list)
        for odt_id, demand_info in self.demand_dict.items():
             demands_by_dest[demand_info['des']].append(odt_id)

        # 2. Add Connection (Arr -> Dep) and Sink (Arr -> Sink) Edges
        print("Adding connection and sink edges...")
        connection_count = 0
        sink_edge_count = 0
        for flight1_id, flight1 in self.flight_data.items():
            arrival_node = (flight1_id, 'arr')
            arrival_time = flight1['arr_time']
            arrival_airport = flight1['des']

            # a) Sink Edges: Connect flight arrival to relevant demand sinks
            if arrival_airport in demands_by_dest:
                 for odt_id in demands_by_dest[arrival_airport]:
                     # This flight arrives at the destination needed by odt_id
                     sink_node = ('sink', odt_id)
                     self._add_edge(arrival_node, sink_node, weight=0, is_flight=False)
                     sink_edge_count += 1

            # b) Connection Edges: Connect flight arrival to potential next flights
            if arrival_airport in self.flights_by_origin:
                # Iterate through flights departing from arrival_airport (sorted by dep_time)
                for flight2_id in self.flights_by_origin[arrival_airport]:
                    flight2 = self.flight_data[flight2_id]
                    departure_time = flight2['dep_time']
                    connection_wait_time = departure_time - arrival_time

                    # Check minimum connection time
                    if connection_wait_time >= self.min_connect_mins:
                        departure_node = (flight2_id, 'dep')
                        self._add_edge(arrival_node, departure_node, weight=connection_wait_time, is_flight=False)
                        connection_count += 1
                    # else: Flight departs too soon for connection

                    # Optimization idea (Optional): If flights are sorted by dep_time,
                    # and connection_wait_time exceeds a reasonable MAX_CONNECT_MINS,
                    # we could potentially 'break' early for this arrival_node.
                    # However, without an explicit MAX limit, we check all possibilities.
        print(f"Added {connection_count} connection edges.")
        print(f"Added {sink_edge_count} sink edges.")

        # 3. Add Source Edges (Source -> Dep)
        print("Adding source edges...")
        source_edge_count = 0
        for odt_id, demand in self.demand_dict.items():
            source_node = ('source', odt_id)
            origin_airport = demand['ori']
            ready_time = demand['time']

            if origin_airport in self.flights_by_origin:
                # Iterate flights from origin (sorted by dep_time)
                for flight_id in self.flights_by_origin[origin_airport]:
                    flight = self.flight_data[flight_id]
                    departure_time = flight['dep_time']

                    # Connect if flight departs at or after ready time
                    if departure_time >= ready_time:
                        initial_wait_time = departure_time - ready_time
                        departure_node = (flight_id, 'dep')
                        self._add_edge(source_node, departure_node, weight=initial_wait_time, is_flight=False)
                        source_edge_count += 1
                    # else: Flight departs before demand is ready

                    # Optimization: Since flights are sorted, if the current flight
                    # departs *much* later than ready_time, subsequent flights will
                    # depart even later. Could break if wait time exceeds a threshold,
                    # but connecting to all valid first flights is safer for finding
                    # the true K shortest paths.
            else: # Handle case where no flights leave from the demand origin
                 print(f"Warning: No flights found departing from origin '{origin_airport}' for ODT '{odt_id}'. No paths possible.")
                 pass # Dijkstra will handle this by finding no path from source

        print(f"Added {source_edge_count} source edges.")
        end_time = timer.time()
        print(f"Graph building completed in {end_time - start_time:.2f} seconds.")
        # Estimate node count (sources + sinks + 2*flights)
        est_nodes = len(self.demand_dict) * 2 + len(self.flight_data) * 2
        print(f"Graph nodes estimated around: {est_nodes}")


    def _dijkstra(self, start_node, end_node, forbidden_edges=None, forbidden_nodes=None):
        """
        Finds the shortest path using Dijkstra's algorithm. (Modified for Yen's)
        Uses a tie-breaker for heapq stability.
        """
        global tie_breaker # Use the global counter

        if forbidden_edges is None: forbidden_edges = set()
        if forbidden_nodes is None: forbidden_nodes = set()

        # Priority Queue: (cost, tie_breaker_id, current_node, path_list)
        # path_list stores: [{'node': node_tuple, 'edge_info': edge_dict_or_None}, ...]
        pq = [(0.0, next(tie_breaker), start_node, [{'node': start_node, 'edge_info': None}])]
        # visited_costs stores the minimum cost found *so far* to reach a node
        visited_costs = defaultdict(lambda: float('inf'))
        visited_costs[start_node] = 0.0

        while pq:
            current_cost, _, current_node, current_path_list = heapq.heappop(pq)

            # Optimization: If we already found a shorter path to this node, skip
            if current_cost > visited_costs[current_node]:
                continue

            # Goal reached
            if current_node == end_node:
                return current_cost, current_path_list

            # Explore neighbors
            if current_node not in self.graph: # Node might not have outgoing edges (e.g., sink)
                continue

            for edge in self.graph[current_node]:
                neighbor = edge['neighbor']
                weight = edge['weight']
                edge_flight_id = edge['flight_id'] # Integer or None

                # --- Check Forbidden Nodes and Edges ---
                if neighbor in forbidden_nodes:
                    continue

                # Edge representation for forbidden check: (from_node, to_node, flight_id_or_None)
                edge_representation = (current_node, neighbor, edge_flight_id)
                if edge_representation in forbidden_edges:
                    continue
                # --- End Forbidden Check ---

                new_cost = current_cost + weight

                # Relaxation: If this is a shorter path to neighbor
                if new_cost < visited_costs[neighbor]:
                    visited_costs[neighbor] = new_cost
                    # Create the next step in the path list, storing the edge *leading* to the neighbor
                    new_step = {'node': neighbor, 'edge_info': edge}
                    new_path_list = current_path_list + [new_step]
                    heapq.heappush(pq, (new_cost, next(tie_breaker), neighbor, new_path_list))

        return float('inf'), None # No path found


    def _format_path_output(self, path_list, total_cost):
        """Formats the raw path list from Dijkstra/Yen's into the desired output."""
        if not path_list or len(path_list) < 2: # Need source and at least one more node
            return None

        flight_ids = []
        min_capacity = float('inf')

        # Iterate through the path steps to find flight segments
        # path_list = [{'node': n0, 'edge': None}, {'node': n1, 'edge': e01}, {'node': n2, 'edge': e12}, ...]
        for step in path_list[1:]: # Skip the source node which has no incoming edge_info
            edge_info = step['edge_info']
            if edge_info and edge_info['is_flight']:
                flight_id = edge_info['flight_id']
                capacity = edge_info['capacity']
                if flight_id is not None: # Should always be true if is_flight
                    flight_ids.append(flight_id)
                    # Ensure capacity is finite for comparison
                    current_cap = capacity if math.isfinite(capacity) else float('inf')
                    min_capacity = min(min_capacity, current_cap)

        # If no flights were taken, min_capacity remains 'inf'. Set to 0 or None? Using 0.
        if min_capacity == float('inf'):
            min_capacity = 0.0

        return {
            'total_time': total_cost, # Time from ready_time to final arrival
            'flight_ids': flight_ids, # List of integer flight IDs
            'min_capacity': min_capacity
        }


    def find_k_shortest_paths(self):
        """
        Finds the K shortest paths for each demand ODT using Yen's algorithm.
        """
        print(f"\nFinding {self.k_paths} shortest paths for {len(self.demand_dict)} demands...")
        all_paths_results = {}
        demand_count = 0
        total_demands = len(self.demand_dict)
        start_run_time = timer.time()

        for odt_id, demand in self.demand_dict.items():
            demand_count += 1
            if demand_count % 100 == 0: # Progress indicator
                elapsed = timer.time() - start_run_time
                print(f"  Processed {demand_count}/{total_demands} demands... Elapsed: {elapsed:.2f}s")

            start_node = ('source', odt_id)
            end_node = ('sink', odt_id)   # Unique sink for this demand

            A = [] # Stores final K paths: (cost, path_list)
            B = [] # Candidate paths heap: (cost, tie_breaker, path_list)

            # 1. Find the first shortest path
            cost1, path_list1 = self._dijkstra(start_node, end_node)

            if path_list1:
                A.append((cost1, path_list1))
                # Don't add to B yet, B is for *deviations* from paths in A
            else:
                all_paths_results[odt_id] = [] # No paths found
                continue

            # 2. Iterate to find paths k = 2 to K
            for k in range(1, self.k_paths):
                if k > len(A): # Cannot find k if k-1 doesn't exist
                    break

                # Get the (k-1)th shortest path (index k-1)
                prev_cost, prev_path_list = A[k-1]

                # Iterate through nodes in the (k-1)th path to find spur points
                # Path structure: [{'node':n0,'edge':None},{'node':n1,'edge':e01},...]
                for i in range(len(prev_path_list) - 1): # Spur node cannot be the sink
                    spur_node_info = prev_path_list[i]
                    spur_node = spur_node_info['node']

                    # Root path: segment of the (k-1)th path up to spur_node
                    root_path_list = prev_path_list[:i+1]
                    # Calculate cost of the root path by summing edge weights
                    root_cost = sum(step['edge_info']['weight'] for step in root_path_list[1:] if step['edge_info'])

                    forbidden_edges_yen = set()
                    forbidden_nodes_yen = set()

                    # Forbid nodes in the root path (excluding spur node itself)
                    for step in root_path_list[:-1]:
                         forbidden_nodes_yen.add(step['node'])

                    # Forbid the edge *leaving* the spur node if a previous path in A (0..k-1)
                    # shares the same root path.
                    edge_leaving_spur_in_prev = prev_path_list[i+1]['edge_info'] if (i+1 < len(prev_path_list)) else None

                    for path_idx in range(k): # Check paths A[0] to A[k-1]
                         cost_j, path_list_j = A[path_idx]
                         # Check if path_j is long enough and shares the same root *node sequence*
                         if len(path_list_j) > i + 1:
                             path_j_root_nodes = [step['node'] for step in path_list_j[:i+1]]
                             root_path_nodes = [step['node'] for step in root_path_list]
                             if path_j_root_nodes == root_path_nodes:
                                 # Paths share the root. Forbid the edge leaving the spur node in path_j.
                                 edge_info_j = path_list_j[i+1]['edge_info']
                                 if edge_info_j:
                                     forbidden_edge_j = (spur_node, edge_info_j['neighbor'], edge_info_j['flight_id'])
                                     forbidden_edges_yen.add(forbidden_edge_j)


                    # --- Calculate spur path ---
                    spur_cost_delta, spur_path_segment_list = self._dijkstra(spur_node, end_node,
                                                                             forbidden_edges=forbidden_edges_yen,
                                                                             forbidden_nodes=forbidden_nodes_yen)

                    # If a valid spur path is found from spur_node to end_node
                    if spur_path_segment_list:
                        # Combine root and spur paths
                        # spur_path_segment starts with spur_node (edge=None), which is the last node of root_path
                        # Concatenate root_path with spur_path[1:]
                        combined_path_list = root_path_list + spur_path_segment_list[1:]
                        total_new_cost = root_cost + spur_cost_delta

                        # Add the potential new path to the candidate heap B
                        heapq.heappush(B, (total_new_cost, next(tie_breaker), combined_path_list))


                # Select the best path from B that isn't already in A
                found_next_path = False
                while B:
                    potential_cost, _, potential_path_list = heapq.heappop(B)

                    # Check if this path (by node sequence) is already in A
                    is_in_A = False
                    potential_nodes = tuple(item['node'] for item in potential_path_list)
                    for cost_a, path_a in A:
                         if tuple(item['node'] for item in path_a) == potential_nodes:
                            is_in_A = True
                            break

                    if not is_in_A:
                         # Found the k-th shortest path
                         A.append((potential_cost, potential_path_list))
                         found_next_path = True
                         break # Move to find the (k+1)th path

                # If B is exhausted or no new paths found, stop for this demand
                if not found_next_path:
                    break

            # Format the results for this ODT
            formatted_paths = []
            A.sort(key=lambda x: x[0]) # Ensure final paths are sorted by cost
            for cost, path_list in A:
                 formatted = self._format_path_output(path_list, cost)
                 if formatted:
                     formatted_paths.append(formatted)

            all_paths_results[odt_id] = formatted_paths

        end_run_time = timer.time()
        print(f"\nFinished finding K shortest paths in {end_run_time - start_run_time:.2f} seconds.")
        return all_paths_results

# --- Script Execution ---

# 1. Read Data
print("Reading data...")
schedule_df = read_capacity(CAPACITY_FILE, airport_substitutions=airport_substitutions)
demand_df = read_market(MARKET_FILE, airport_substitutions=airport_substitutions)
print(demand_df.head()) # Display sample demand data

# 2. Initialize Network and Build Graph
# Check if dataframes are valid before proceeding
if not schedule_df.empty and not demand_df.empty:
    try:
        flight_network = FlightNetwork(schedule_df, demand_df, K_PATHS, MIN_CONNECT_MINS)

        # 3. Find K Shortest Paths
        k_shortest_paths_results = flight_network.find_k_shortest_paths()

        # 4. Output Results (Example for first few demands)
        print("\n--- K Shortest Paths Results (Sample) ---")
        output_count = 0
        for odt, paths in k_shortest_paths_results.items():
            if output_count < 10: # Print details only for the first 10 demands
                demand_info = flight_network.demand_dict.get(odt, {})
                print(f"ODT: {odt} (Demand: {demand_info.get('ori','?')}-{demand_info.get('des','?')} @ {demand_info.get('time','?')})")
                if paths:
                    for i, path in enumerate(paths):
                        print(f"  Path {i+1}:")
                        print(f"    Total Time (mins): {path['total_time']:.2f}")
                        # Optional: Print flight details
                        flight_details_str = []
                        for fid in path['flight_ids']:
                             f_info = flight_network.flight_data.get(fid) # fid is int
                             if f_info:
                                 flight_details_str.append(f"{fid}({f_info['ori']}-{f_info['des']}@{f_info['dep_time']})")
                             else:
                                 flight_details_str.append(f"{fid}(?)")
                        print(f"    Flights: {' -> '.join(flight_details_str)}")
                        print(f"    Min Capacity (kg): {path['min_capacity']:.2f}") # Format capacity
                else:
                    print("  No paths found.")
                print("-" * 20)
            elif output_count == 10:
                 print("\n... (output truncated for brevity) ...")
                 break # Stop printing samples after 10
            output_count += 1

        print(f"\nFound path results for {len(k_shortest_paths_results)} ODTs.")
        # The 'k_shortest_paths_results' dictionary contains all the results.

    except ValueError as e:
        print(f"\nError during FlightNetwork initialization: {e}")
    except Exception as e:
        print(f"\nAn unexpected error occurred during processing: {e}")
        import traceback
        traceback.print_exc() # Print detailed traceback for debugging
else:
    print("\nError: Failed to read valid schedule or demand data. Cannot proceed.")

Reading data...
Capacity data read: 2057 rows.
Market data read: 3185 rows.
   ori  des     demand  time           ODT
0  AMS  LAX  4366.6250  1080  AMS/LAX/1080
1  AMS  LAX  2481.6250  2520  AMS/LAX/2520
2  AMS  LAX  3985.1125  3960  AMS/LAX/3960
3  AMS  LAX  3415.7500  5400  AMS/LAX/5400
4  AMS  LAX  3933.5000  6840  AMS/LAX/6840

Initializing Flight Network...
Precomputing flight data lookups...
Building graph edges...
Adding flight leg edges...
Added 2057 flight leg edges.
Adding connection and sink edges...
Added 96571 connection edges.
Added 121331 sink edges.
Adding source edges...
Added 99076 source edges.
Graph building completed in 0.31 seconds.
Graph nodes estimated around: 10484

Finding 3 shortest paths for 3185 demands...
  Processed 100/3185 demands... Elapsed: 0.90s
  Processed 200/3185 demands... Elapsed: 2.12s
  Processed 300/3185 demands... Elapsed: 3.34s
  Processed 400/3185 demands... Elapsed: 4.72s
  Processed 500/3185 demands... Elapsed: 6.66s
  Processed 600/318